# Data preparation

## Table of Contents
- [Introduction](#Introduction)
- [Data sources](#Data-sources)
- [Libraries](#Libraries)
- [Preparing folders structure](#Preparing-folders-structure)
- [Player injuries data preparation](#Player-injuries-data-preparation)
    * [Seasons Filtering](#Seasons-Filtering)
    * [Clubs Filtering](#Clubs-Filtering)
- [Player performances preparation](#Player-performances-preparation)
    * [2016-17 season](#2016-17-season)
    * [2017-18 season](#2017-18-season)
    * [2018-19 season](#2018-19-season)
    * [2019-20 season](#2019-20-season)
    * [2020-21 season](#2020-21-season)
    * [2021-22 season](#2021-22-season)
    * [2022-23 season](#2022-23-season)
    * [2023-24 season](#2023-24-season)
    * [Names correction](#Names-correction)
- [Matches data preparation](#Matches-data-preparation)
- [Encoding](#Encoding)
    * [Matches encoding](#Matches-encoding)
    * [Players performances encoding](#Players-performances-encoding)
      

## Introduction
This notebook is used to prepare the data for the project. The data is taken from various sources and needs to be cleaned and prepared for further use.

## Data sources
The data is taken from the following sources:
- **Football Matches:** Historical match data is sourced from the [Football Data](https://www.football-data.co.uk/matches.php) website, which compiles extensive match statistics from various leagues. For this project, we focus specifically on data from the Premier League.


- **Players Performances:** For player performance analytics, I utilize data from [Vaastav's GitHub repository for Fantasy Premier League](https://github.com/vaastav/Fantasy-Premier-League). This dataset provides detailed performance statistics for Premier League players for each match they participated in.


- **Players Injuries:** The dataset for players injuries has been taken from [Yasirbm's GitHub repository](https://github.com/yasirbm/soccer_player_injury_model/blob/main/data/injuries.csv). It contains information about the injuries players suffered during the seasons.


### Libraries

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from datetime import datetime
from fuzzywuzzy import fuzz
from collections import deque

## Preparing folders structure
In `remove_files` function I remove the files that are no longer needed. The function takes the following parameters: `path`, `file_names`, and `directory_names`. The `path` parameter is the path to the folder where the files are located. The `file_names` parameter is a list of file names that need to be removed. The `directory_names` parameter is a list of directory names that need to be removed. The function uses the `os` and `shutil` libraries to remove the files and directories.

In [2]:
def remove_files(path, file_names, directory_names):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file in file_names:
                os.remove(os.path.join(root, file))
        for directory_name in directory_names:
            if directory_name in dirs:
                shutil.rmtree(os.path.join(root, directory_name))
                
remove_files('data', ['history.csv','players_raw.csv', 'raw.json','fixtures.csv','fbref_overview.csv','id_dict.csv','player_idlist.csv','best_players.csv','cleaned_players.csv'], ['managers', 'understat','fbref'])

## Player injuries data preparation
In this seciton I prepare the data for the player injuries. The data is taken from the [Vaastav's GitHub repository](https://github.com/vaastav/Fantasy-Premier-League).

In [3]:
injuries_df = pd.read_csv('data/injuries.csv', encoding='ISO-8859-1')

In [4]:
injuries_df.head(20)

,player_name,player_url,season_injured,injury,injured_since,injured_until,duration,games_missed,club
0,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,22/23,Adductor problems,2023-03-26,2023-03-30,4 days,NaN,Real Madrid
1,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,22/23,Hamstring Injury,2023-02-06,2023-02-17,11 days,3.0,Real Madrid
2,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,22/23,Sciatic Problem,2022-09-29,2022-10-17,18 days,5.0,Real Madrid
3,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,21/22,Pubitis,2022-05-30,2022-06-30,31 days,NaN,Real Madrid
4,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,21/22,Corona virus,2021-12-30,2022-01-03,4 days,1.0,Real Madrid
5,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,20/21,Minor Knock,2020-10-05,2020-10-15,10 days,NaN,Real Madrid
6,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,19/20,Muscle Injury,2020-03-09,2020-03-26,17 days,NaN,Chelsea FC
7,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,19/20,Gastric problems,2019-10-03,2019-10-07,4 days,1.0,Chelsea FC
8,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,19/20,Bone buckling,2019-07-29,2019-08-05,7 days,NaN,Chelsea FC
9,\n \n ...,https://www.transfermarkt.com/thibaut-courtois...,18/19,Minor Knock,2019-03-28,2019-03-28,0 days,NaN,Real Madrid


The `player_name` column contains the player's name in not suitable format so i will clean it.

In [5]:
injuries_df['player_name'] = injuries_df['player_name'].str.replace('#\d+', '').str.split('\n').str[-1].str.strip()

In [6]:
injuries_df.head()

,player_name,player_url,season_injured,injury,injured_since,injured_until,duration,games_missed,club
0,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Adductor problems,2023-03-26,2023-03-30,4 days,NaN,Real Madrid
1,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Hamstring Injury,2023-02-06,2023-02-17,11 days,3.0,Real Madrid
2,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Sciatic Problem,2022-09-29,2022-10-17,18 days,5.0,Real Madrid
3,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,21/22,Pubitis,2022-05-30,2022-06-30,31 days,NaN,Real Madrid
4,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,21/22,Corona virus,2021-12-30,2022-01-03,4 days,1.0,Real Madrid


### Seasons Filtering
Find the unique values in the `season_injured` column. Keep only the seasons I have the matches and player performances data for.

In [7]:
injuries_df['season_injured'].unique()

array(['22/23', '21/22', '20/21', '19/20', '18/19', '17/18', '16/17',
       '15/16', '14/15', '13/14', nan, '09/10', '12/13', '11/12', '08/09',
       '07/08', '10/11', '06/07', '03/04', '05/06'], dtype=object)

In [8]:
seasons = ['22/23', '21/22', '20/21', '19/20', '18/19', '17/18', '16/17']
injuries_df = injuries_df[injuries_df['season_injured'].isin(seasons)]

In [9]:
injuries_df['season_injured'].unique()

array(['22/23', '21/22', '20/21', '19/20', '18/19', '17/18', '16/17'],
      dtype=object)

### Clubs Filtering
Filter only the clubs that are in the Premier League. Because the prediction model will be based on the Premier League data. Also map the clubs names to the short format which is used in the other datasets.

In [10]:
injuries_df['club'].unique()

array(['Real Madrid', 'Chelsea FC', 'AFC Bournemouth', 'Stoke City',
       'AC Horsens', 'Arsenal FC', 'Paris Saint-Germain', 'FC Barcelona',
       'Borussia Mönchengladbach', 'Crystal Palace', 'West Ham United',
       'Manchester City', 'Bristol City', 'Fulham FC', 'CD Leganés',
       'Alanyaspor', 'Deportivo Alavés', 'Reading FC', 'Vitesse Arnhem',
       'Aston Villa', 'ACF Fiorentina', 'RCD Mallorca', 'FC Schalke 04',
       'SpVgg Greuther Fürth', 'Olympique Marseille',
       'West Bromwich Albion', 'Zenit St. Petersburg', 'FC Groningen',
       'Torino FC', 'Leicester City', 'AS Roma', 'Manchester United',
       'Watford FC', 'AS Monaco', 'PSV Eindhoven', 'AC Milan',
       'Trabzonspor', 'FC Sochi', 'Juventus FC', 'Udinese Calcio',
       'Spartak Moscow', 'Inter Milan', 'Fenerbahce',
       'CD Universidad Católica', 'Austria Vienna', 'Shanghai Port',
       'Atalanta BC', 'Rio Ave FC', 'FK Partizan Belgrade',
       'FK Cukaricki', 'Excelsior Rotterdam',
       'Anzhi Ma

In [11]:
# Mapping the clubs to the short names
clubs_mapping = {"Tottenham Hotspur": "Spurs",
           "Liverpool FC": "Liverpool",
           "Southampton FC": "Southampton",
           "West Ham United": "West Ham",
           "Stoke City": "Stoke",
           "Arsenal FC": "Arsenal",
           "Manchester United": "Man Utd",
           "Watford FC": "Watford",
           "AFC Bournemouth": "Bournemouth",
           "Leicester City": "Leicester",
           "Swansea City": "Swansea",
           "West Bromwich Albion": "West Brom",
           "Everton FC": "Everton",
           "Hull City": "Hull",
           "Crystal Palace": "Crystal Palace",
           "Middlesbrough FC": "Middlesbrough",
           "Sunderland AFC": "Sunderland",
           "Burnley FC": "Burnley",
           "Manchester City": "Man City",
           "Huddersfield Town": "Huddersfield",
           "Newcastle United": "Newcastle",
           "Brighton & Hove Albion": "Brighton",
           "Wolverhampton Wanderers": "Wolves",
           "Sheffield United": "Sheffield Utd",
           "Leeds United": "Leeds",
           "Fulham FC": "Fulham",
           "Aston Villa": "Aston Villa",
           "Norwich City": "Norwich",
           "Brentford FC": "Brentford",
           "Nottingham Forest": "Nott'm Forest",
           "Luton Town": "Luton"}

injuries_df['club'] = injuries_df['club'].map(clubs_mapping)


In [12]:
injuries_df['club'].unique()

array([nan, 'Bournemouth', 'Stoke', 'Arsenal', 'Crystal Palace',
       'West Ham', 'Man City', 'Fulham', 'Aston Villa', 'West Brom',
       'Leicester', 'Man Utd', 'Watford', 'Newcastle', 'Liverpool',
       'Everton', 'Swansea', 'Spurs', 'Southampton', 'Middlesbrough',
       "Nott'm Forest", 'Brentford', 'Brighton', 'Sunderland',
       'Sheffield Utd', 'Luton', 'Burnley', 'Norwich', 'Hull', 'Leeds',
       'Huddersfield', 'Wolves'], dtype=object)

In [13]:
injuries_df.head(20)

,player_name,player_url,season_injured,injury,injured_since,injured_until,duration,games_missed,club
0,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Adductor problems,2023-03-26,2023-03-30,4 days,NaN,NaN
1,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Hamstring Injury,2023-02-06,2023-02-17,11 days,3.0,NaN
2,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,22/23,Sciatic Problem,2022-09-29,2022-10-17,18 days,5.0,NaN
3,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,21/22,Pubitis,2022-05-30,2022-06-30,31 days,NaN,NaN
4,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,21/22,Corona virus,2021-12-30,2022-01-03,4 days,1.0,NaN
5,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,20/21,Minor Knock,2020-10-05,2020-10-15,10 days,NaN,NaN
6,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,19/20,Muscle Injury,2020-03-09,2020-03-26,17 days,NaN,NaN
7,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,19/20,Gastric problems,2019-10-03,2019-10-07,4 days,1.0,NaN
8,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,19/20,Bone buckling,2019-07-29,2019-08-05,7 days,NaN,NaN
9,Thibaut Courtois,https://www.transfermarkt.com/thibaut-courtois...,18/19,Minor Knock,2019-03-28,2019-03-28,0 days,NaN,NaN


In [14]:
injuries_df = injuries_df.drop(columns=['player_url'])

In [15]:
injuries_df.head()

,player_name,season_injured,injury,injured_since,injured_until,duration,games_missed,club
0,Thibaut Courtois,22/23,Adductor problems,2023-03-26,2023-03-30,4 days,NaN,NaN
1,Thibaut Courtois,22/23,Hamstring Injury,2023-02-06,2023-02-17,11 days,3.0,NaN
2,Thibaut Courtois,22/23,Sciatic Problem,2022-09-29,2022-10-17,18 days,5.0,NaN
3,Thibaut Courtois,21/22,Pubitis,2022-05-30,2022-06-30,31 days,NaN,NaN
4,Thibaut Courtois,21/22,Corona virus,2021-12-30,2022-01-03,4 days,1.0,NaN


Filter out the rows where the `club` column is missing.

In [16]:
injuries_df['club'].isna().sum()

4178

Drop the rows where the `club` column is missing. This is necessary because the club information is important for the prediction model.

In [17]:
injuries_df = injuries_df.dropna(subset=['club'])

In [18]:
injuries_df['club'].isna().sum()

0

In [19]:
injuries_df.shape

(3905, 8)

Format the `season_injured` column to the format used in the other datasets.

In [20]:
def reformat_season(season):
    parts = season.split('/')
    return '20' + parts[0] + '-' + '20' + parts[1]

In [21]:
injuries_df['season_injured'] = injuries_df['season_injured'].apply(reformat_season)

In [22]:
injuries_df['season_injured'].unique()

array(['2020-2021', '2018-2019', '2019-2020', '2022-2023', '2021-2022',
       '2017-2018', '2016-2017'], dtype=object)

Save the cleaned data to a new file.

In [23]:
injuries_df.to_csv('data/player_injuries.csv', index=False)

In [24]:
injuries = pd.read_csv('data/player_injuries.csv')

# Player performances preparation

This part aims to prepare the player performances data. This data is taken from the [Vaastav's GitHub repository for Fantasy Premier League](https://github.com/vaastav/Fantasy-Premier-League). In this part a lot of unexpected issues are encountered and need to be resolved iteratively. This is the reason why i use the iterative approach for data preparation (prepare seasons one by one). My meaning was to prepare all seasons at once using merged data and functions but because the data was scraped they are not consistent and i need to prepare them one by one.

In [25]:
merged_seasons_performances = pd.read_csv('data/merged_seasons.csv', low_memory=False)

In [26]:
merged_seasons_performances.head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,...,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,...,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,...,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Abdul Rahman Baba,DEF,NaN,0,0,0,0,0.0,80,...,2.0,0.0,0,0,0,0,55,True,0,1


We can see that in the `merged_seasons_performances` dataframe in column `season_x` we have the seasons `2016-17`, `2017-18`, `2020-21`, `2021-22`, `2022-23`, `2023-24`.
So we need to find way to find data for the seasons `2018-19` and `2019-20`.

In [27]:
merged_seasons_performances['season_x'].unique()

array(['2016-17', '2017-18', '2020-21', '2021-22', '2022-23', '2023-24'],
      dtype=object)

For seasons `2018-19` and `2019-20` we take data from the `merged_gw_2018_19.csv` and `merged_gw_2019_20.csv` files. 

In [28]:
performance_2018_19 = pd.read_csv('data/2018_19/gws/merged_gw_2018_19.csv', encoding='ISO-8859-1')
performance_2019_20 = pd.read_csv('data/2019_20/gws/merged_gw_2019_20.csv', encoding='ISO-8859-1')

Find the shape of the dataframes. We can see that the dataframes have different shapes.

In [29]:
performance_2018_19.shape

(21790, 56)

In [30]:
performance_2019_20.shape

(22560, 33)

In [31]:
merged_seasons_performances.shape

(126076, 37)

Find the common columns in the dataframes.

In [32]:
common_columns = merged_seasons_performances.columns.intersection(performance_2018_19.columns).intersection(
    performance_2019_20.columns)
common_columns

Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [33]:
performance_2018_19['season_x'] = '2018-19'
performance_2019_20['season_x'] = '2019-20'

In [34]:
common_columns = merged_seasons_performances.columns.intersection(performance_2018_19.columns).intersection(
    performance_2019_20.columns)
common_columns

Index(['season_x', 'name', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [35]:
merged_seasons_performances.columns

Index(['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW'],
      dtype='object')

After finding the intersection columns in the dataframes we can merge them.

In [36]:
players_performance_metrics = ['season_x', 'name', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW']

performance_2018_19 = performance_2018_19[players_performance_metrics]
performance_2019_20 = performance_2019_20[players_performance_metrics]

In [37]:
# Merge the dataframes
merged_players_performances = pd.concat([performance_2018_19, performance_2019_20, merged_seasons_performances])

In [38]:
merged_players_performances.shape

(170426, 37)

Now we can see that all seasons are in the `merged_players_performances` dataframe.

In [39]:
merged_players_performances['season_x'].unique()

array(['2018-19', '2019-20', '2016-17', '2017-18', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [40]:
merged_players_performances['opp_team_name'].unique()

array([nan, 'Chelsea', 'Spurs', 'Liverpool', 'Southampton', 'West Ham',
       'Stoke', 'Arsenal', 'Man Utd', 'Watford', 'Bournemouth',
       'Leicester', 'Swansea', 'West Brom', 'Everton', 'Hull',
       'Crystal Palace', 'Middlesbrough', 'Sunderland', 'Burnley',
       'Man City', 'Huddersfield', 'Newcastle', 'Brighton', 'Wolves',
       'Sheffield Utd', 'Leeds', 'Fulham', 'Aston Villa', 'Norwich',
       'Brentford', "Nott'm Forest", 'Luton'], dtype=object)

The `opp_team_name` column contains the names of the teams in not suitable format so i will clean it.

In [41]:
merged_players_performances['opp_team_name'] = merged_players_performances['opp_team_name'].replace('Man Utd', 'Man United')

merged_players_performances['opp_team_name'] = merged_players_performances['opp_team_name'].replace('Spurs', 'Tottenham')

In [42]:
def convert_kickoff_time(df):
    df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])
    df['Date'] = df['kickoff_time'].apply(lambda x: x.strftime('%d/%m/%y'))
    
    return df

In [43]:
merged_players_performances = convert_kickoff_time(merged_players_performances)

In [44]:
merged_players_performances.sort_values(by=['Date','name'], inplace=True)

In [45]:
merged_players_performances.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
2210,2016-17,Aaron Ramsey,0,0,4,0,2.7,16,181,0,...,478,198,76,True,0,19,MID,NaN,Crystal Palace,01/01/17
2211,2016-17,Aaron Wan-Bissaka,0,0,0,0,0.0,612,181,0,...,7,8,45,False,0,19,DEF,NaN,Arsenal,01/01/17
2212,2016-17,Abdoulaye Doucouré,0,0,10,0,5.3,482,190,4,...,7,10,46,True,0,19,MID,NaN,Tottenham,01/01/17
2219,2016-17,Ainsley Maitland-Niles,0,0,0,0,0.0,593,181,0,...,3,7,45,True,0,19,DEF,NaN,Crystal Palace,01/01/17
2220,2016-17,Alex Iwobi,0,3,37,1,62.2,21,181,0,...,6051,1559,58,True,0,19,MID,NaN,Crystal Palace,01/01/17


In [46]:
merged_players_performances['team_x'].isna().sum()

64384

As we can see in the merged dataframe many records have the `team_x` column missing. This is because the player is not in the team at the time of the match. This is very important attribute, so we need to find a way to fill the missing values. We have the data with the matches and date and time of the matches. Also, we have the teams that were playing the match. In the merged_players_performances dataframe we have the `opp_team_name` column which is the team that the player was playing against and also the `kickoff_time` column which is the time of the match. So we can use this information to find the team of the player. For this purpose i created the `find_player_team` function.

In [47]:
def find_player_team(player_df, match_df):
    
    merged_away = pd.merge(player_df, match_df, how='left',
                           left_on=['Date', 'opp_team_name'],
                           right_on=['Date', 'AwayTeam'])
    
    merged_home = pd.merge(player_df.reset_index(drop=True), match_df.reset_index(drop=True), how='left',
                       left_on=['Date', 'opp_team_name'],
                       right_on=['Date', 'HomeTeam'])
    # If HomeTeam in merged_home is not NaN => the player belongs to the AwayTeam because opp_team_name is 'HomeTeam'
    
    # If AwayTeam in merged_away is not NaN => the player belongs to the HomeTeam because opp_team_name is AwayTeam

    player_df = player_df.copy()
    player_df.loc[:, 'team'] = np.where(pd.notna(merged_home['HomeTeam']), merged_home['AwayTeam'],
                                    np.where(pd.notna(merged_away['AwayTeam']), merged_away['HomeTeam'], np.nan))

    return player_df


# Check and convert the date format
def check_and_convert(df):
    new_dates = []
    for date in df['Date']:
        try:
            dt = datetime.strptime(date, '%d/%m/%Y')
            new_date = datetime.strftime(dt, '%d/%m/%y')
            new_dates.append(new_date)
        except ValueError:
            new_dates.append(date)
    df['Date'] = new_dates
    return df

## 2016-17 season
First we need to find the matches for the 2016-17 season. Apply the `check_and_convert` function which will convert the date format to the format used in the other datasets. Then apply the `find_player_team` function to find the team of the player.

In [48]:
players_performances_2016_2017 = merged_players_performances[merged_players_performances['season_x'] == '2016-17']

matches_2016_2017 = check_and_convert(pd.read_csv('data/2016_17/matches_2016_17.csv'))
players_performances_2016_2017_with_team = find_player_team(players_performances_2016_2017, matches_2016_2017)

Check if there are missing values in the `team` column.

In [49]:
players_performances_2016_2017_with_team['team'].isna().sum()

0

For safety reasons run the `dropna` function to remove the missing values.

In [50]:
players_performances_2016_2017_with_team.dropna(subset=['team'], inplace=True)

In [51]:
players_performances_2016_2017_with_team

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date,team
2210,2016-17,Aaron Ramsey,0,0,4,0,2.7,16,181,0,...,198,76,True,0,19,MID,NaN,Crystal Palace,01/01/17,Arsenal
2211,2016-17,Aaron Wan-Bissaka,0,0,0,0,0.0,612,181,0,...,8,45,False,0,19,DEF,NaN,Arsenal,01/01/17,Crystal Palace
2212,2016-17,Abdoulaye Doucouré,0,0,10,0,5.3,482,190,4,...,10,46,True,0,19,MID,NaN,Tottenham,01/01/17,Watford
2219,2016-17,Ainsley Maitland-Niles,0,0,0,0,0.0,593,181,0,...,7,45,True,0,19,DEF,NaN,Crystal Palace,01/01/17,Arsenal
2220,2016-17,Alex Iwobi,0,3,37,1,62.2,21,181,0,...,1559,58,True,0,19,MID,NaN,Crystal Palace,01/01/17,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,2016-17,Wes Morgan,0,0,22,1,0.6,167,185,0,...,28560,48,True,0,19,DEF,NaN,West Ham,31/12/16,Leicester
2432,2016-17,Willian Borges Da Silva,0,3,43,0,18.3,86,183,2,...,3529,71,True,0,19,MID,NaN,Stoke,31/12/16,Chelsea
2433,2016-17,Willy Caballero,0,0,0,0,0.0,219,186,0,...,165,47,False,0,19,GK,NaN,Liverpool,31/12/16,Man City
2434,2016-17,Winston Reid,0,0,20,0,0.3,453,185,1,...,2536,51,False,0,19,DEF,NaN,Leicester,31/12/16,West Ham


In [52]:
players_performances_2016_2017_with_team.shape

(8689, 39)

Save the data to a csv file.

In [53]:
players_performances_2016_2017_with_team.to_csv('data/2016_17/players_performances_2016_17.csv', index=False)

## 2017-18 season
First we need to find the matches for the 2017-18 season. Apply the `check_and_convert` function which will convert the date format to the format used in the other datasets. Then apply the `find_player_team` function to find the team of the player.

In [54]:
players_performances_2017_2018 = merged_players_performances[merged_players_performances['season_x'] == '2017-18']

matches_2017_2018 = check_and_convert(pd.read_csv('data/2017_18/matches_2017_18.csv'))
players_performances_2017_2018_with_team = find_player_team(players_performances_2017_2018, matches_2017_2018)

Check if there are missing values in the `team` column.

In [55]:
players_performances_2017_2018_with_team['team'].isna().sum()

0

For safety reasons run the `dropna` function to remove the missing values.

In [56]:
players_performances_2017_2018_with_team.dropna(subset=['team'], inplace=True)

In [57]:
players_performances_2017_2018_with_team

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date,team
12716,2017-18,Aaron Lennon,0,0,3,0,0.4,153,214,1,...,336,54,True,0,22,MID,NaN,Man United,01/01/18,Everton
12717,2017-18,Aaron Mooy,0,0,12,0,8.8,172,215,3,...,29144,54,False,0,22,MID,NaN,Leicester,01/01/18,Huddersfield
12718,2017-18,Aaron Ramsdale,0,0,0,0,0.0,549,212,0,...,364,40,False,0,22,GK,NaN,Brighton,01/01/18,Bournemouth
12722,2017-18,Adam Lallana,0,0,14,1,48.9,228,213,0,...,270,73,False,0,22,MID,NaN,Burnley,01/01/18,Liverpool
12723,2017-18,Adam Smith,1,2,29,0,27.6,35,212,2,...,466,48,False,0,22,DEF,NaN,Brighton,01/01/18,Bournemouth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12690,2017-18,Shkodran Mustafi,0,0,16,0,5.4,12,210,1,...,5779,54,False,1,21,DEF,NaN,West Brom,31/12/17,Arsenal
12695,2017-18,Theo Walcott,0,0,0,0,0.0,15,210,0,...,468,71,False,0,21,MID,NaN,West Brom,31/12/17,Arsenal
12696,2017-18,Timothy Fosu-Mensah,0,0,26,1,1.0,269,203,0,...,1218,44,True,0,21,DEF,NaN,Man City,31/12/17,Crystal Palace
12706,2017-18,Wayne Hennessey,0,0,31,1,0.0,115,203,0,...,2073,43,True,0,21,GK,NaN,Man City,31/12/17,Crystal Palace


In [58]:
players_performances_2017_2018_with_team.shape

(11345, 39)

In [59]:
players_performances_2017_2018_with_team.to_csv('data/2017_18/players_performances_2017_18.csv', index=False)

## 2018-19 season


In [60]:
players_performances_2018_2019 = merged_players_performances[merged_players_performances['season_x'] == '2018-19']

matches_2018_2019 = check_and_convert(pd.read_csv('data/2018_19/matches_2018_19.csv'))

In [61]:
players_performances_2018_2019.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
10964,2018-19,Aaron_Ramsey_14,0,0,21,0,1.6,14,201,0,...,2690,1622,72,True,0,21,NaN,NaN,NaN,01/01/19
10969,2018-19,Aboubakar_Kamara_189,0,0,25,0,2.0,189,201,2,...,54466,49890,44,False,0,21,NaN,NaN,NaN,01/01/19
10976,2018-19,Ademola_Lookman_170,0,0,0,0,0.0,170,205,0,...,9,158,51,True,0,21,NaN,NaN,NaN,01/01/19
10978,2018-19,Adrien Sebastian_Perruchet Silva_231,0,0,0,0,0.0,231,205,0,...,83,136,47,False,0,21,NaN,NaN,NaN,01/01/19
10980,2018-19,Ainsley_Maitland-Niles_20,0,0,14,0,3.9,20,201,1,...,8349,1955,44,True,0,21,NaN,NaN,NaN,01/01/19


In [62]:
players_performances_2018_2019['team_x'].isna().sum()

21790

For this season i found that we dont have player team which is also in other datasets but here we have missing values in `opp_team_name`, that means we cannot find the team for the players in thw way we did for the previous seasons. So we need to find another way to find the team for the players.

In [63]:
players_performances_2018_2019['opp_team_name'].isna().sum()

21790

Load raw players data for the 2018-19 season.

In [64]:
players_raw_2018_2019 = pd.read_csv('data/2019_20/players_raw_2019_20.csv')

In [65]:
players_raw_2018_2019

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,2,2,242,0.0,0.0,4,69140,0,0,-4,...,58,43,25007,0,47630,0,0.3,8.4,Mustafi,2
1,0,4,204,100.0,100.0,4,98745,0,0,-3,...,81,44,206616,0,159819,0,0.3,8.5,Bellerín,2
2,2,1,331,100.0,100.0,4,111457,0,0,-3,...,98,55,65194,0,134275,0,0.5,10.6,Kolasinac,4
3,2,3,244,100.0,100.0,3,154043,0,0,-5,...,119,41,610816,0,653555,0,0.1,9.1,Maitland-Niles,4
4,0,5,305,100.0,100.0,4,39476,0,0,-2,...,76,57,182201,0,231413,0,0.0,11.9,Sokratis,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0,0,0,NaN,NaN,0,179456,0,0,0,...,189,0,12504,0,7078,0,0.0,0.0,Buur,0
662,0,0,0,0.0,0.0,0,232351,0,0,-1,...,181,0,21912,0,11421,0,0.0,0.0,Giles,0
663,0,0,4,NaN,NaN,0,428610,0,0,0,...,246,1,1913,0,1146,0,0.0,0.2,Jordao,0
664,1,2,74,NaN,NaN,3,200600,0,0,-6,...,124,25,8281,0,3461,0,0.6,5.1,Podence,0


This data have `team` column which is the code of the team. We need to find the team name for given code. So I found the unique teams in the `team` column and print two players from each team to find the team name. Then I manually check where the players are playing in this season and map the team names to the codes.

In [66]:
unique_teams = players_raw_2018_2019['team'].unique()

for team in unique_teams:
    players_from_team = players_raw_2018_2019[players_raw_2018_2019['team'] == team]
    two_players = players_from_team[['first_name', 'second_name']][:2]

    for index, player in two_players.iterrows():
        print(f'Team: {team}, Player: {player["first_name"]} {player["second_name"]}')


Team: 1, Player: Shkodran Mustafi
Team: 1, Player: Héctor Bellerín
Team: 2, Player: Ahmed El Mohamady
Team: 2, Player: James Chester
Team: 3, Player: Steve Cook
Team: 3, Player: Nathan Aké
Team: 4, Player: Shane Duffy
Team: 4, Player: Martín Montoya
Team: 5, Player: Ben Mee
Team: 5, Player: James Tarkowski
Team: 6, Player: Marcos Alonso
Team: 6, Player: Antonio Rüdiger
Team: 7, Player: Patrick van Aanholt
Team: 7, Player: James Tomkins
Team: 8, Player: Alex Iwobi
Team: 8, Player: Lucas Digne
Team: 9, Player: Ricardo Domingos Barbosa Pereira
Team: 9, Player: Benjamin Chilwell
Team: 10, Player: Andrew Robertson
Team: 10, Player: Trent Alexander-Arnold
Team: 11, Player: Aymeric Laporte
Team: 11, Player: Kyle Walker
Team: 12, Player: Aaron Wan-Bissaka
Team: 12, Player: Harry Maguire
Team: 13, Player: Matt Ritchie
Team: 13, Player: Fabian Schär
Team: 14, Player: Timm Klose
Team: 14, Player: Ben Godfrey
Team: 15, Player: Lys Mousset
Team: 15, Player: Enda Stevens
Team: 16, Player: Jannik Ves

Create dataframe with the teams.

In [68]:
data = {
    'code': [i for i in range(1, 21)],
    'team': ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
             'Chelsea', 'Crystal Palace', 'Everton', 'Leicester', 'Liverpool',
             'Man City', 'Man Utd', 'Newcastle', 'Norwich', 'Sheffield Utd',
             'Southampton', 'Spurs', 'Watford', 'West Ham', 'Wolves']
}

teams_2018_2019 = pd.DataFrame(data)

In [69]:
teams_2018_2019

,code,team
0,1,Arsenal
1,2,Aston Villa
2,3,Bournemouth
3,4,Brighton
4,5,Burnley
5,6,Chelsea
6,7,Crystal Palace
7,8,Everton
8,9,Leicester
9,10,Liverpool


Save the teams data to a csv file.

In [70]:
teams_2018_2019.to_csv('data/2018_19/teams.csv', index=False)

In [71]:
teams_2018_2019 = pd.read_csv('data/2018_19/teams.csv')

In [72]:
players_raw_2018_2019.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,2,2,242,0.0,0.0,4,69140,0,0,-4,...,58,43,25007,0,47630,0,0.3,8.4,Mustafi,2
1,0,4,204,100.0,100.0,4,98745,0,0,-3,...,81,44,206616,0,159819,0,0.3,8.5,Bellerín,2
2,2,1,331,100.0,100.0,4,111457,0,0,-3,...,98,55,65194,0,134275,0,0.5,10.6,Kolasinac,4
3,2,3,244,100.0,100.0,3,154043,0,0,-5,...,119,41,610816,0,653555,0,0.1,9.1,Maitland-Niles,4
4,0,5,305,100.0,100.0,4,39476,0,0,-2,...,76,57,182201,0,231413,0,0.0,11.9,Sokratis,6


Create `name` column which is the concatenation of `first_name` and `second_name` columns. Add the team name to the players' data.

In [73]:
player_teams_2018_2019 = players_raw_2018_2019[['first_name', 'second_name', 'team']].copy()
player_teams_2018_2019.loc[:, 'name'] = player_teams_2018_2019['first_name'] + ' ' + player_teams_2018_2019[
    'second_name']

player_teams_2018_2019 = pd.merge(player_teams_2018_2019, teams_2018_2019, how='left', left_on='team', right_on='code')

In [74]:
player_teams_2018_2019

,first_name,second_name,team_x,name,code,team_y
0,Shkodran,Mustafi,1,Shkodran Mustafi,1,Arsenal
1,Héctor,Bellerín,1,Héctor Bellerín,1,Arsenal
2,Sead,Kolasinac,1,Sead Kolasinac,1,Arsenal
3,Ainsley,Maitland-Niles,1,Ainsley Maitland-Niles,1,Arsenal
4,Sokratis,Papastathopoulos,1,Sokratis Papastathopoulos,1,Arsenal
...,...,...,...,...,...,...
661,Oskar,Buur,20,Oskar Buur,20,Wolves
662,Ryan,Giles,20,Ryan Giles,20,Wolves
663,Bruno André,Cavaco Jordao,20,Bruno André Cavaco Jordao,20,Wolves
664,Daniel,Castelo Podence,20,Daniel Castelo Podence,20,Wolves


Keep only the `name` and `team` columns.

In [75]:
player_teams_2018_2019 = player_teams_2018_2019.rename(columns={'team_y': 'team'})
player_teams_2018_2019 = player_teams_2018_2019[['name', 'team']]

In [76]:
player_teams_2018_2019

,name,team
0,Shkodran Mustafi,Arsenal
1,Héctor Bellerín,Arsenal
2,Sead Kolasinac,Arsenal
3,Ainsley Maitland-Niles,Arsenal
4,Sokratis Papastathopoulos,Arsenal
...,...,...
661,Oskar Buur,Wolves
662,Ryan Giles,Wolves
663,Bruno André Cavaco Jordao,Wolves
664,Daniel Castelo Podence,Wolves


In [77]:
player_teams_2018_2019.to_csv('data/2018_19/player_teams_2018_19.csv', index=False)

Load the merged data for processing.

In [78]:
merged_qw_2018_19 = pd.read_csv('data/2018_19/gws/merged_gw_2018_19.csv', encoding='ISO-8859-1')

In [79]:
merged_qw_2018_19.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell_402,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,55,False,0,0,1
1,Aaron_Lennon_83,0,22,0,1,0,6,1,1,17,...,17.0,3,0,0,0,50,False,0,0,1
2,Aaron_Mooy_199,0,51,0,0,0,24,0,2,40,...,0.0,2,0,0,0,55,True,0,0,1
3,Aaron_Ramsey_14,0,11,0,0,0,7,0,0,7,...,9.0,1,0,0,0,75,True,0,0,1
4,Aaron_Wan-Bissaka_145,1,29,1,0,3,38,1,11,19,...,0.0,12,0,0,0,40,False,0,0,1


As we can see the `name` column contains the player names in not suitable format so i will clean it. For example from `Aaron_Cresswell_402` we want to get `Aaron Cresswell`.

In [80]:
merged_qw_2018_19['name'] = merged_qw_2018_19['name'].str.replace('\d+', '', regex=True).str.replace('_', ' ')

In [81]:
merged_qw_2018_19['name'].unique()

array(['Aaron Cresswell ', 'Aaron Lennon ', 'Aaron Mooy ',
       'Aaron Ramsey ', 'Aaron Wan-Bissaka ', 'Abdelhamid Sabiri ',
       'Abdoulaye Doucouré ', 'Aboubakar Kamara ', 'Adama Diakhaby ',
       'Adama Traoré ', 'Adam Lallana ', 'Adam Masina ', 'Adam Smith ',
       'Ademola Lookman ', 'Adrian Mariappa ',
       'Adrien Sebastian Perruchet Silva ',
       'Adrián San Miguel del Castillo ', 'Ainsley Maitland-Niles ',
       'Alberto Moreno ', 'Aleksandar Mitrovic ', 'Alexander Sørloth ',
       'Alexandre Lacazette ', 'Alexis Sánchez ', 'Alex Iwobi ',
       'Alex McCarthy ', 'Alex Oxlade-Chamberlain ', 'Alex Pritchard ',
       'Alex Smithies ', 'Alfie Mawson ', 'Alireza Jahanbakhsh ',
       'Alisson Ramses Becker ', 'Anders Lindegaard ', 'Ander Herrera ',
       'Andreas Christensen ', 'Andreas Pereira ', 'Andrew Robertson ',
       'Andrew Surman ', 'Andre Gray ', 'Andriy Yarmolenko ',
       'Andros Townsend ', 'André Filipe Tavares Gomes ',
       'André-Frank Zambo Angui

After printing the unique values I found there is whitespace at the end of the names so i will remove it.

In [82]:
merged_qw_2018_19['name'] = merged_qw_2018_19['name'].str.strip()

In [83]:
merged_qw_2018_19['name'].unique()

array(['Aaron Cresswell', 'Aaron Lennon', 'Aaron Mooy', 'Aaron Ramsey',
       'Aaron Wan-Bissaka', 'Abdelhamid Sabiri', 'Abdoulaye Doucouré',
       'Aboubakar Kamara', 'Adama Diakhaby', 'Adama Traoré',
       'Adam Lallana', 'Adam Masina', 'Adam Smith', 'Ademola Lookman',
       'Adrian Mariappa', 'Adrien Sebastian Perruchet Silva',
       'Adrián San Miguel del Castillo', 'Ainsley Maitland-Niles',
       'Alberto Moreno', 'Aleksandar Mitrovic', 'Alexander Sørloth',
       'Alexandre Lacazette', 'Alexis Sánchez', 'Alex Iwobi',
       'Alex McCarthy', 'Alex Oxlade-Chamberlain', 'Alex Pritchard',
       'Alex Smithies', 'Alfie Mawson', 'Alireza Jahanbakhsh',
       'Alisson Ramses Becker', 'Anders Lindegaard', 'Ander Herrera',
       'Andreas Christensen', 'Andreas Pereira', 'Andrew Robertson',
       'Andrew Surman', 'Andre Gray', 'Andriy Yarmolenko',
       'Andros Townsend', 'André Filipe Tavares Gomes',
       'André-Frank Zambo Anguissa', 'André Schürrle', 'Andy Carroll',
       '

Find the differences between the names in the `player_teams_2018_2019` and `merged_qw_2018_19` dataframes and correct them. This is important because we need to merge the dataframes on the `name` column.

In [84]:
names_2018_2019 = set(player_teams_2018_2019['name'].unique())

names_merged = set(merged_qw_2018_19['name'].unique())

diff_2018_2019_and_merged = names_2018_2019 - names_merged

diff_merged_and_2018_2019 = names_merged - names_2018_2019

print('Names unique to player_teams_2018_2019:', diff_2018_2019_and_merged)
print('Names unique to merged_qw_2018_19:', diff_merged_and_2018_2019)

Names unique to player_teams_2018_2019: {'Lukas Rupp', 'Moise Kean', 'Emil Krafth', 'Jack Robinson', 'Daniel James', 'Ignacio Pussetto', 'Jake Eastwood', 'Daniel Ceballos Fernández', 'João Pedro Cavaco Cancelo', 'James Daly', 'Luke Thomas', 'Sander Berge', 'Dennis Srbeny', 'Tom Trybull', 'Ben Godfrey', 'Ralf Fahrmann', 'Jack Stacey', 'Marco Stiepermann', 'Bernardo Costa Da Rosa', 'Ryan Giles', 'Craig Dawson', 'Gabriel Teodoro Martinelli Silva', 'Tomas Soucek', 'Kieran Tierney', 'Kaine Hayden', 'Pedro Lomba Neto', 'Jordan Thomas', 'Grant Hanley', 'Jeremy Ngakia', 'Pablo Marí', 'Mario Vrancic', 'Jean-Philippe Gbamin', 'Jack Grealish', 'Dean Henderson', 'Jack Rodwell', 'Armando Broja', 'Axel Tuanzebe', 'David McGoldrick', 'Jack Young', 'Emiliano Buendía', 'Owen Otasowie', 'Valentino Lazaro', 'Nicolas Pépé', 'Ismaïla Sarr', 'Emile Smith Rowe', 'Joelinton Cássio Apolinário de Lira', 'Reece James', 'Leon Clarke', 'Adam Idah', 'Caoimhin Kelleher', 'William Smallbone', 'Lukas Jensen', 'Nathan 

In [85]:
players_names_mapping = {
    'David De Gea': 'David de Gea',
    'Caglar Söyüncü': 'Çaglar Söyüncü',
}

merged_qw_2018_19['name'] = merged_qw_2018_19['name'].replace(players_names_mapping)

Check the differences again after the correction.

In [86]:
names_2019_2020 = set(player_teams_2018_2019['name'].unique())
names_merged = set(merged_qw_2018_19['name'].unique())

diff_2019_2020_and_merged = names_2019_2020 - names_merged

diff_merged_and_2019_2020 = names_merged - names_2019_2020

print('Names unique to player_teams_2019_2020:', diff_2019_2020_and_merged)
print('Names unique to merged_qw_2019_20:', diff_merged_and_2019_2020)

Names unique to player_teams_2019_2020: {'Lukas Rupp', 'Moise Kean', 'Emil Krafth', 'Jack Robinson', 'Daniel James', 'Ignacio Pussetto', 'Jake Eastwood', 'Daniel Ceballos Fernández', 'João Pedro Cavaco Cancelo', 'James Daly', 'Luke Thomas', 'Sander Berge', 'Dennis Srbeny', 'Tom Trybull', 'Ben Godfrey', 'Ralf Fahrmann', 'Jack Stacey', 'Marco Stiepermann', 'Bernardo Costa Da Rosa', 'Ryan Giles', 'Craig Dawson', 'Gabriel Teodoro Martinelli Silva', 'Tomas Soucek', 'Kieran Tierney', 'Kaine Hayden', 'Pedro Lomba Neto', 'Jordan Thomas', 'Grant Hanley', 'Jeremy Ngakia', 'Pablo Marí', 'Mario Vrancic', 'Jean-Philippe Gbamin', 'Jack Grealish', 'Dean Henderson', 'Jack Rodwell', 'Armando Broja', 'Axel Tuanzebe', 'David McGoldrick', 'Jack Young', 'Emiliano Buendía', 'Owen Otasowie', 'Valentino Lazaro', 'Nicolas Pépé', 'Ismaïla Sarr', 'Emile Smith Rowe', 'Joelinton Cássio Apolinário de Lira', 'Reece James', 'Leon Clarke', 'Adam Idah', 'Caoimhin Kelleher', 'William Smallbone', 'Lukas Jensen', 'Nathan 

Merge the dataframes on the `name` column.

In [87]:
players_performances_2018_2019 = merged_qw_2018_19.merge(player_teams_2018_2019, how='left', on='name')

In [88]:
players_performances_2018_2019['Date'] = pd.to_datetime(players_performances_2018_2019.loc[:, 'kickoff_time']).dt.strftime('%d/%m/%y')

In [89]:
players_performances_2018_2019['Date'].isna().sum()

0

In [90]:
players_performances_2018_2019

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team,Date
0,Aaron Cresswell,0,0,0,0,0,0,0,0,0,...,0,0,0,55,False,0,0,1,West Ham,12/08/18
1,Aaron Lennon,0,22,0,1,0,6,1,1,17,...,0,0,0,50,False,0,0,1,Burnley,12/08/18
2,Aaron Mooy,0,51,0,0,0,24,0,2,40,...,0,0,0,55,True,0,0,1,Brighton,11/08/18
3,Aaron Ramsey,0,11,0,0,0,7,0,0,7,...,0,0,0,75,True,0,0,1,NaN,12/08/18
4,Aaron Wan-Bissaka,1,29,1,0,3,38,1,11,19,...,0,0,0,40,False,0,0,1,Man Utd,11/08/18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21785,Yves Bissouma,0,21,0,0,0,7,0,5,16,...,30,73,43,46,True,0,0,38,Brighton,12/05/19
21786,Zech Medley,0,0,0,0,0,0,0,0,0,...,62,171,109,40,False,0,0,38,Arsenal,12/05/19
21787,Zeze Steven Sessegnon,0,0,0,0,0,0,0,0,0,...,125,332,207,40,True,0,0,38,NaN,12/05/19
21788,Álvaro Morata,0,0,0,0,0,0,0,0,0,...,-139,15,154,84,False,0,0,38,NaN,12/05/19


In [91]:
players_performances_2018_2019['opponent_team'].isna().sum()

0

In [92]:
teams_2018_2019 = pd.read_csv('data/2018_19/teams.csv')

In [93]:
teams_2018_2019

,code,team
0,1,Arsenal
1,2,Aston Villa
2,3,Bournemouth
3,4,Brighton
4,5,Burnley
5,6,Chelsea
6,7,Crystal Palace
7,8,Everton
8,9,Leicester
9,10,Liverpool


In [94]:
players_performances_2018_2019 = pd.merge(players_performances_2018_2019, teams_2018_2019, left_on='opponent_team', right_on='code', how='left')

In [95]:
players_performances_2018_2019.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team_x,Date,code,team_y
0,Aaron Cresswell,0,0,0,0,0,0,0,0,0,...,0,55,False,0,0,1,West Ham,12/08/18,12,Man Utd
1,Aaron Lennon,0,22,0,1,0,6,1,1,17,...,0,50,False,0,0,1,Burnley,12/08/18,16,Southampton
2,Aaron Mooy,0,51,0,0,0,24,0,2,40,...,0,55,True,0,0,1,Brighton,11/08/18,6,Chelsea
3,Aaron Ramsey,0,11,0,0,0,7,0,0,7,...,0,75,True,0,0,1,NaN,12/08/18,13,Newcastle
4,Aaron Wan-Bissaka,1,29,1,0,3,38,1,11,19,...,0,40,False,0,0,1,Man Utd,11/08/18,9,Leicester


In [96]:
players_performances_2018_2019['team_y'].isna().sum()

0

I found that there is records with missing home team.

In [97]:
players_performances_2018_2019['team_x'].isna().sum()

6579

Split the data into two dataframes. One with missing team and one with the team.

In [98]:
players_performances_2018_2019_with_missing_team = players_performances_2018_2019[players_performances_2018_2019['team_x'].isna()]

players_performances_2018_2019 = players_performances_2018_2019.dropna(subset=['team_x'])

Find the missing team for the players. Use the `find_player_team` function to find the team.

In [99]:
players_performances_2018_2019_with_missing_team = players_performances_2018_2019_with_missing_team.rename(columns={'team_y': 'opp_team_name', 'team_x': 'team'})

In [100]:
matches_2018_2019 = check_and_convert(pd.read_csv('data/2018_19/matches_2018_19.csv'))

In [101]:
players_performances_2018_2019_with_missing_team = find_player_team(players_performances_2018_2019_with_missing_team, matches_2018_2019)

Some players are still have missing team. So we need to find another way to find the team for the players.

In [102]:
players_performances_2018_2019_with_missing_team['team'].isna().sum()

2857

In [103]:
players_performances_2018_2019_with_missing_team['name'].nunique()

197

When I sort the data by the `Date` column I found that the players with missing team have some records with missing `team` and some with the `team`. So I will fill the missing values with the team. So I will fill the missing values with the team where the team is not missing for the same player.

In [104]:
players_performances_2018_2019_with_missing_team

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team,Date,code,opp_team_name
3,Aaron Ramsey,0,11,0,0,0,7,0,0,7,...,0,75,True,0,0,1,NaN,12/08/18,13,Newcastle
5,Abdelhamid Sabiri,0,0,0,0,0,0,0,0,0,...,0,45,True,0,0,1,Huddersfield,11/08/18,6,Chelsea
7,Aboubakar Kamara,0,15,0,0,0,2,0,0,10,...,0,45,True,0,0,1,Fulham,11/08/18,7,Crystal Palace
8,Adama Diakhaby,0,5,0,0,0,2,0,0,5,...,0,50,True,0,0,1,Huddersfield,11/08/18,6,Chelsea
15,Adrien Sebastian Perruchet Silva,0,76,0,0,0,13,0,1,65,...,0,50,False,0,0,1,NaN,10/08/18,14,Norwich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21778,Yannick Bolasie,0,0,0,0,0,0,0,0,0,...,9,58,False,0,0,38,NaN,12/05/19,17,Spurs
21781,Yohan Benalouane,0,0,0,0,0,0,0,0,0,...,450,39,True,0,0,38,Leicester,12/05/19,6,Chelsea
21783,Younes Kaboul,0,0,0,0,0,0,0,0,0,...,17,42,True,0,0,38,Watford,12/05/19,19,West Ham
21787,Zeze Steven Sessegnon,0,0,0,0,0,0,0,0,0,...,207,40,True,0,0,38,NaN,12/05/19,15,Sheffield Utd


In [105]:
unique_players = players_performances_2018_2019_with_missing_team.dropna(subset=['team']).drop_duplicates('name')[['name', 'team']]

players_performances_2018_2019_with_missing_team = pd.merge(players_performances_2018_2019_with_missing_team, unique_players, on='name', how='left')

players_performances_2018_2019_with_missing_team['team'] = players_performances_2018_2019_with_missing_team['team_x'].where(players_performances_2018_2019_with_missing_team['team_x'].notna(), players_performances_2018_2019_with_missing_team['team_y'])

players_performances_2018_2019_with_missing_team = players_performances_2018_2019_with_missing_team.drop(columns=['team_x', 'team_y'])

In [106]:
players_performances_2018_2019_with_missing_team['team'].isna().sum()

1

In [107]:
players_performances_2018_2019_with_missing_team = players_performances_2018_2019_with_missing_team.dropna(subset=['team'])

In [108]:
players_performances_2018_2019_with_missing_team['team'].isna().sum()

0

In [109]:
players_performances_2018_2019 = players_performances_2018_2019.rename(columns={'team_y': 'opp_team_name', 'team_x': 'team'})

In [110]:
players_performances_2018_2019 = pd.concat([players_performances_2018_2019, players_performances_2018_2019_with_missing_team])

In [111]:
players_performances_2018_2019 = players_performances_2018_2019.drop(columns='code')

In [112]:
players_performances_2018_2019.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team,Date,opp_team_name
0,Aaron Cresswell,0,0,0,0,0,0,0,0,0,...,0,0,55,False,0,0,1,West Ham,12/08/18,Man Utd
1,Aaron Lennon,0,22,0,1,0,6,1,1,17,...,0,0,50,False,0,0,1,Burnley,12/08/18,Southampton
2,Aaron Mooy,0,51,0,0,0,24,0,2,40,...,0,0,55,True,0,0,1,Brighton,11/08/18,Chelsea
4,Aaron Wan-Bissaka,1,29,1,0,3,38,1,11,19,...,0,0,40,False,0,0,1,Man Utd,11/08/18,Leicester
6,Abdoulaye Doucouré,0,66,0,0,0,16,1,1,55,...,0,0,60,True,0,0,1,Watford,11/08/18,Bournemouth


Add season attribute with the value `2018-19`.

In [113]:
players_performances_2018_2019['season_x'] = '2018-19'

In [114]:
players_performances_2018_2019.to_csv('data/2018_19/players_performances_2018_19.csv', index=False)

In [115]:
players_performances_2018_2019_with_team = pd.read_csv('data/2018_19/players_performances_2018_19.csv')

In [116]:
players_performances_2018_2019_with_team['team'].isna().sum()

0

In [117]:
players_performances_2018_2019_with_team[players_performances_2018_2019_with_team['team'].isna()]

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team,Date,opp_team_name,season_x


In [118]:
players_performances_2018_2019_with_team

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,...,transfers_out,value,was_home,winning_goals,yellow_cards,GW,team,Date,opp_team_name,season_x
0,Aaron Cresswell,0,0,0,0,0,0,0,0,0,...,0,55,False,0,0,1,West Ham,12/08/18,Man Utd,2018-19
1,Aaron Lennon,0,22,0,1,0,6,1,1,17,...,0,50,False,0,0,1,Burnley,12/08/18,Southampton,2018-19
2,Aaron Mooy,0,51,0,0,0,24,0,2,40,...,0,55,True,0,0,1,Brighton,11/08/18,Chelsea,2018-19
3,Aaron Wan-Bissaka,1,29,1,0,3,38,1,11,19,...,0,40,False,0,0,1,Man Utd,11/08/18,Leicester,2018-19
4,Abdoulaye Doucouré,0,66,0,0,0,16,1,1,55,...,0,60,True,0,0,1,Watford,11/08/18,Bournemouth,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21784,Yannick Bolasie,0,0,0,0,0,0,0,0,0,...,9,58,False,0,0,38,Everton,12/05/19,Spurs,2018-19
21785,Yohan Benalouane,0,0,0,0,0,0,0,0,0,...,450,39,True,0,0,38,Leicester,12/05/19,Chelsea,2018-19
21786,Younes Kaboul,0,0,0,0,0,0,0,0,0,...,17,42,True,0,0,38,Watford,12/05/19,West Ham,2018-19
21787,Zeze Steven Sessegnon,0,0,0,0,0,0,0,0,0,...,207,40,True,0,0,38,Fulham,12/05/19,Sheffield Utd,2018-19


## 2019-20 season

In [119]:
players_performances_2019_2020 = merged_players_performances[merged_players_performances['season_x'] == '2019-20']

matches_2019_2020 = check_and_convert(pd.read_csv('data/2019_20/matches_2019_20.csv'))

In [120]:
players_performances_2019_2020.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
11353,2019-20,Aaron_Connolly_534,0,0,-1,0,0.6,534,202,0,...,6408,15499,45,True,0,21,NaN,NaN,NaN,01/01/20
11471,2019-20,Aaron_Cresswell_376,0,0,19,1,23.1,376,210,0,...,13662,8569,47,True,1,21,NaN,NaN,NaN,01/01/20
11235,2019-20,Aaron_Lennon_430,0,0,0,0,0.0,430,203,0,...,14,207,47,True,0,21,NaN,NaN,NaN,01/01/20
11344,2019-20,Aaron_Mooy_516,0,0,11,0,31.3,516,202,1,...,31047,4870,48,True,0,21,NaN,NaN,NaN,01/01/20
11199,2019-20,Aaron_Ramsdale_494,0,0,17,0,0.0,494,210,4,...,36678,25221,47,False,0,21,NaN,NaN,NaN,01/01/20


In [121]:
players_performances_2019_2020['team_x'].isna().sum()

22560

This data have `team` column which is the code of the team. We need to find the team name for given code. Instead of previous season where we don't have teams data here we have it so we can directly map the team names to the codes.

In [122]:
players_performances_2019_2020['opp_team_name'].isna().sum() 

22560

In [123]:
teams_2019_2020 = pd.read_csv('data/2019_20/teams.csv')

In [124]:
teams_2019_2020

,code,draw,form,id,loss,name,played,points,position,pulse_id,...,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,NaN,1,0,Arsenal,0,0,0,1,...,4,1170,1170,1200,1150,1240,1180,NaN,False,0
1,7,0,NaN,2,0,Aston Villa,0,0,0,2,...,2,980,970,1040,1000,1050,1020,NaN,False,0
2,91,0,NaN,3,0,Bournemouth,0,0,0,127,...,2,1030,990,1050,1000,1020,1020,NaN,False,0
3,36,0,NaN,4,0,Brighton,0,0,0,131,...,2,1100,1100,1030,1040,1010,1050,NaN,False,0
4,90,0,NaN,5,0,Burnley,0,0,0,43,...,3,1070,1130,1110,970,1180,1110,NaN,False,0
5,8,0,NaN,6,0,Chelsea,0,0,0,4,...,4,1240,1240,1250,1220,1230,1220,NaN,False,0
6,31,0,NaN,7,0,Crystal Palace,0,0,0,6,...,3,1180,1080,1040,1040,1120,1130,NaN,False,0
7,11,0,NaN,8,0,Everton,0,0,0,7,...,3,1210,1100,1180,1170,1240,1100,NaN,False,0
8,13,0,NaN,9,0,Leicester,0,0,0,26,...,3,1080,1150,1100,1210,1160,1210,NaN,False,0
9,14,0,NaN,10,0,Liverpool,0,0,0,10,...,5,1320,1310,1330,1310,1350,1340,NaN,False,0


In [125]:
players_raw_2019_2020 = pd.read_csv('data/2019_20/players_raw_2019_20.csv')

In [126]:
players_raw_2019_2020

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,2,2,242,0.0,0.0,4,69140,0,0,-4,...,58,43,25007,0,47630,0,0.3,8.4,Mustafi,2
1,0,4,204,100.0,100.0,4,98745,0,0,-3,...,81,44,206616,0,159819,0,0.3,8.5,Bellerín,2
2,2,1,331,100.0,100.0,4,111457,0,0,-3,...,98,55,65194,0,134275,0,0.5,10.6,Kolasinac,4
3,2,3,244,100.0,100.0,3,154043,0,0,-5,...,119,41,610816,0,653555,0,0.1,9.1,Maitland-Niles,4
4,0,5,305,100.0,100.0,4,39476,0,0,-2,...,76,57,182201,0,231413,0,0.0,11.9,Sokratis,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0,0,0,NaN,NaN,0,179456,0,0,0,...,189,0,12504,0,7078,0,0.0,0.0,Buur,0
662,0,0,0,0.0,0.0,0,232351,0,0,-1,...,181,0,21912,0,11421,0,0.0,0.0,Giles,0
663,0,0,4,NaN,NaN,0,428610,0,0,0,...,246,1,1913,0,1146,0,0.0,0.2,Jordao,0
664,1,2,74,NaN,NaN,3,200600,0,0,-6,...,124,25,8281,0,3461,0,0.6,5.1,Podence,0


Create `name` column which is the concatenation of `first_name` and `second_name` columns. Add the team name to the players' data.

In [127]:
player_teams_2019_2020 = players_raw_2019_2020[['first_name', 'second_name', 'team_code']].copy()
player_teams_2019_2020.loc[:, 'name'] = player_teams_2019_2020['first_name'] + ' ' + player_teams_2019_2020[
    'second_name']

player_teams_2019_2020 = pd.merge(player_teams_2019_2020, teams_2019_2020, how='left', left_on='team_code', right_on='code')

player_teams_2019_2020 = player_teams_2019_2020[['name_x', 'name_y']].rename(
    columns={'name_x': 'name', 'name_y': 'team'})


In [128]:
player_teams_2019_2020

,name,team
0,Shkodran Mustafi,Arsenal
1,Héctor Bellerín,Arsenal
2,Sead Kolasinac,Arsenal
3,Ainsley Maitland-Niles,Arsenal
4,Sokratis Papastathopoulos,Arsenal
...,...,...
661,Oskar Buur,Wolves
662,Ryan Giles,Wolves
663,Bruno André Cavaco Jordao,Wolves
664,Daniel Castelo Podence,Wolves


In [129]:
player_teams_2019_2020.to_csv('data/2019_20/player_teams_2019_20.csv', index=False)

In [130]:
player_teams_2019_2020 = pd.read_csv('data/2019_20/player_teams_2019_20.csv')

In [131]:
player_teams_2019_2020.head()

,name,team
0,Shkodran Mustafi,Arsenal
1,Héctor Bellerín,Arsenal
2,Sead Kolasinac,Arsenal
3,Ainsley Maitland-Niles,Arsenal
4,Sokratis Papastathopoulos,Arsenal


In [132]:
merged_qw_2019_20 = pd.read_csv('data/2019_20/gws/merged_gw_2019_20.csv', encoding='ISO-8859-1')

In [133]:
merged_qw_2019_20.head()

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,0.0,0.0,0,0,0,0,50,True,0,1
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,...,3.0,0.0,1,0,0,0,50,True,0,1
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,...,0.0,0.0,0,0,0,0,50,False,0,1
3,Aaron_Ramsdale_494,0,0,11,0,0.0,494,2,1,0,...,1.0,0.0,2,0,0,0,45,True,0,1
4,Aaron_Wan-Bissaka_122,0,2,34,1,16.1,122,9,0,0,...,4.0,2.0,8,0,0,0,55,True,0,1


As we can see the `name` column contains the player names in not suitable format so i will clean it. For example from `Aaron_Cresswell_376` we want to get `Aaron Cresswell`.


In [134]:
merged_qw_2019_20['name'] = merged_qw_2019_20['name'].str.replace('\d+', '', regex=True).str.replace('_', ' ')

In [135]:
merged_qw_2019_20.head()

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,0.0,0.0,0,0,0,0,50,True,0,1
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,3.0,0.0,1,0,0,0,50,True,0,1
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,0.0,0.0,0,0,0,0,50,False,0,1
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,1.0,0.0,2,0,0,0,45,True,0,1
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,4.0,2.0,8,0,0,0,55,True,0,1


In [136]:
merged_qw_2019_20['name'].unique()

array(['Aaron Cresswell ', 'Aaron Lennon ', 'Aaron Mooy ',
       'Aaron Ramsdale ', 'Aaron Wan-Bissaka ', 'Abdoulaye DoucourÃ© ',
       'Adama TraorÃ© ', 'Adam Lallana ', 'Adam Masina ', 'Adam Smith ',
       'Adam Webster ', 'Ademola Lookman ', 'Adrian Mariappa ',
       'AdriÃ¡n San Miguel del Castillo ', 'Ahmed El Mohamady ',
       'Ainsley Maitland-Niles ', 'Albian Ajeti ', 'Alexander Tettey ',
       'Alexandre Nascimento Costa Silva ', 'Alexandre Lacazette ',
       'Alexis SÃ¡nchez ', 'Alex Iwobi ', 'Alex McCarthy ',
       'Alex Oxlade-Chamberlain ', 'Alireza Jahanbakhsh ',
       'Alisson Ramses Becker ', 'Allan Saint-Maximin ',
       'Andreas Christensen ', 'Andreas Pereira ', 'Andrew Robertson ',
       'Andrew Surman ', 'Andre Gray ', 'Andre Green ',
       'Andriy Yarmolenko ', 'Andros Townsend ',
       'AndrÃ© Filipe Tavares Gomes ', 'Andy Carroll ', 'Andy King ',
       'Angelo Ogbonna ', 'Angel Gomes ', 'Angus Gunn ',
       'Anthony Knockaert ', 'Anthony Martial '

After printing the unique values I found there is whitespace at the end of the names so i will remove it.

In [137]:
merged_qw_2019_20['name'] = merged_qw_2019_20['name'].str.strip()

In [138]:
player_teams_2019_2020['name'].unique()

array(['Shkodran Mustafi', 'Héctor Bellerín', 'Sead Kolasinac',
       'Ainsley Maitland-Niles', 'Sokratis Papastathopoulos',
       'Nacho Monreal', 'Laurent Koscielny', 'Konstantinos Mavropanos',
       'Carl Jenkinson', 'Rob Holding', 'Pierre-Emerick Aubameyang',
       'Alexandre Lacazette', 'Edward Nketiah', 'Bernd Leno',
       'Mesut Özil', 'Henrikh Mkhitaryan', 'Granit Xhaka',
       'Lucas Torreira', 'Mohamed Elneny', 'Matteo Guendouzi',
       'David Luiz Moreira Marinho', 'Emiliano Martínez',
       'Calum Chambers', 'Daniel Ceballos Fernández', 'Cédric Soares',
       'Nicolas Pépé', 'Reiss Nelson', 'Joseph Willock',
       'Gabriel Teodoro Martinelli Silva', 'Kieran Tierney',
       'Bukayo Saka', 'Emile Smith Rowe', 'Tyreece John-Jules',
       'Pablo Marí', 'Zech Medley', 'Matt Macey', 'Matthew Smith',
       'Ahmed El Mohamady', 'James Chester', 'Neil Taylor',
       'Kortney Hause', 'Jonathan Kodjia', 'Ørjan Nyland',
       'Conor Hourihane', 'Jack Grealish', 'Anwar El

Find the differences between the names in the `player_teams_2019_2020` and `merged_qw_2019_20` dataframes and correct them. This is important because we need to merge the dataframes on the `name` column.

In [139]:
names_2019_2020 = set(player_teams_2019_2020['name'].unique())
names_merged = set(merged_qw_2019_20['name'].unique())

diff_2019_2020_and_merged = names_2019_2020 - names_merged

diff_merged_and_2019_2020 = names_merged - names_2019_2020

print('Names unique to player_teams_2019_2020:', diff_2019_2020_and_merged)
print('Names unique to merged_qw_2019_20:', diff_merged_and_2019_2020)

Names unique to player_teams_2019_2020: {'Martín Montoya', 'Muhamed Bešić', 'Sébastien Haller', 'Rúben Gonçalo Silva Nascimento Vinagre', 'Adalberto Peñaranda', 'Sergio Agüero', 'Sadio Mané', 'Leroy Sané', 'Rui Pedro dos Santos Patrício', 'Miguel Almirón', 'Ilkay Gündogan', 'Onel Hernández', 'Daniel Ceballos Fernández', 'Frédéric Guilbert', 'Nicolás Otamendi', 'João Pedro Cavaco Cancelo', 'Cheikhou Kouyaté', 'Bruno André Cavaco Jordao', 'Ørjan Nyland', 'Javier Hernández Balcázar', 'André Filipe Tavares Gomes', 'Jürgen Locadia', 'Borja González Tomás', 'Alexis Sánchez', 'Pierre-Emile Højbjerg', 'Jose Luis Mato Sanmartín', 'Davinson Sánchez', 'Hélder Costa', 'João Filipe Iria Santos Moutinho', 'José Ignacio Peleteiro Romallo', 'Sebastian Prödl', 'Georges-Kévin Nkoudou', 'Adrián San Miguel del Castillo', 'Raúl Jiménez', 'Rúben Diogo da Silva Neves', 'José Ángel Esmorís Tasende', 'Gonçalo Bento Soares Cardoso', 'Víctor Camarasa', 'Pablo Marí', 'Abdoulaye Doucouré', 'Gaëtan Bong', 'Fabian S

After printing the unique values I found that there are some differences in the names. I will correct them. 

In [140]:
players_names_mapping = {
    "IsmaÃ¯la Sarr": "Ismaïla Sarr",
    "SÃ©bastien Haller": "Sébastien Haller",
    "JosÃ© Holebas": "José Holebas",
    "Pablo MarÃ\xad": "Pablo Marí",
    "Javier HernÃ¡ndez BalcÃ¡zar": "Javier Hernández Balcázar",
    "Abdoulaye DoucourÃ©": "Abdoulaye Doucouré",
    "Emiliano MartÃ\xadnez": "Emiliano Martínez",
    "Francisco FemenÃ\xada Far": "Francisco Femenía Far",
    "Joelinton CÃ¡ssio ApolinÃ¡rio de Lira": "Joelinton Cássio Apolinário de Lira",
    "Mesut Ã\x96zil": "Mesut Özil",
    "Sebastian PrÃ¶dl": "Sebastian Prödl",
    "RÃºben Diogo da Silva Neves": "Rúben Diogo da Silva Neves",
    "Miguel AlmirÃ³n": "Miguel Almirón",
    "Pedro RodrÃ\xadguez Ledesma": "Pedro Rodríguez Ledesma",
    "Jonas LÃ¶ssl": "Jonas Lössl",
    "JosÃ© Reina": "José Reina",
    "FabiÃ¡n Balbuena": "Fabián Balbuena",
    "Pascal GroÃ\x9f": "Pascal Groß",
    "Rui Pedro dos Santos PatrÃ\xadcio": "Rui Pedro dos Santos Patrício",
    "GonÃ§alo Bento Soares Cardoso": "Gonçalo Bento Soares Cardoso",
    "RÃºben GonÃ§alo Silva Nascimento Vinagre": "Rúben Gonçalo Silva Nascimento Vinagre",
    "Alexis SÃ¡nchez": "Alexis Sánchez",
    "FrÃ©dÃ©ric Guilbert": "Frédéric Guilbert",
    "Leroy SanÃ©": "Leroy Sané",
    "JÃ¼rgen Locadia": "Jürgen Locadia",
    "Ilkay GÃ¼ndogan": "Ilkay Gündogan",
    "Daniel Ceballos FernÃ¡ndez": "Daniel Ceballos Fernández",
    "CÃ©dric Soares": "Cédric Soares",
    "JoÃ£o Pedro Junqueira de Jesus": "João Pedro Junqueira de Jesus",
    "Davinson SÃ¡nchez": "Davinson Sánchez",
    "Romain SaÃ¯ss": "Romain Saïss",
    "Sergio AgÃ¼ero": "Sergio Agüero",
    "Georges-KÃ©vin Nkoudou": "Georges-Kévin Nkoudou",
    "AndrÃ© Filipe Tavares Gomes": "André Filipe Tavares Gomes",
    "Adalberto PeÃ±aranda": "Adalberto Peñaranda",
    "Victor LindelÃ¶f": "Victor Lindelöf",
    "JoÃ£o Manuel Neves VirgÃ\xadnia": "João Manuel Neves Virgínia",
    "VÃ\xadctor Camarasa": "Víctor Camarasa",
    "JoÃ£o Filipe Iria Santos Moutinho": "João Filipe Iria Santos Moutinho",
    "Nicolas PÃ©pÃ©": "Nicolas Pépé",
    "GaÃ«tan Bong": "Gaëtan Bong",
    "Davy PrÃ¶pper": "Davy Pröpper",
    "Onel HernÃ¡ndez": "Onel Hernández",
    "Emiliano BuendÃ\xada": "Emiliano Buendía",
    "CÃ©sar Azpilicueta": "César Azpilicueta",
    "Antonio RÃ¼diger": "Antonio Rüdiger",
    "Cheikhou KouyatÃ©": "Cheikhou Kouyaté",
    "JosÃ© Diogo Dalot Teixeira": "José Diogo Dalot Teixeira",
    "Ã\x98rjan Nyland": "Ørjan Nyland",
    "Djibril SidibÃ©": "Djibril Sidibé",
    "MartÃ\xadn Montoya": "Martín Montoya",
    "JosÃ© Ignacio Peleteiro Romallo": "José Ignacio Peleteiro Romallo",
    "HÃ©ctor BellerÃ\xadn": "Héctor Bellerín",
    "JosÃ© Ã\x81ngel EsmorÃ\xads Tasende": "José Ángel Esmorís Tasende",
    "JoÃ£o Pedro Cavaco Cancelo": "João Pedro Cavaco Cancelo",
    "NicolÃ¡s Otamendi": "Nicolás Otamendi",
    "Pierre-Emile HÃ¸jbjerg": "Pierre-Emile Højbjerg",
    "Ayoze PÃ©rez": "Ayoze Pérez",
    "Bernard AnÃ\xadcio Caldeira Duarte": "Bernard Anício Caldeira Duarte",
    "Fabian SchÃ¤r": "Fabian Schär",
    "Federico FernÃ¡ndez": "Federico Fernández",
    "RaÃºl JimÃ©nez": "Raúl Jiménez",
    "Ã\x87aglar SÃ¶yÃ¼ncÃ¼": "Çaglar Söyüncü",
    "Nathan AkÃ©": "Nathan Aké",
    "Carlos SÃ¡nchez": "Carlos Sánchez",
    "JesÃºs Vallejo LÃ¡zaro": "Jesús Vallejo Lázaro",
    "Sadio ManÃ©": "Sadio Mané",
    "Borja GonzÃ¡lez TomÃ¡s": "Borja González Tomás",
    "Jose Luis Mato SanmartÃ\xadn": "Jose Luis Mato Sanmartín",
    "HÃ©lder Costa": "Hélder Costa",
    "Bruno AndrÃ© Cavaco Jordao": "Bruno André Cavaco Jordao",
    "AdriÃ¡n San Miguel del Castillo": "Adrián San Miguel del Castillo",
    "JosÃ© Heriberto Izquierdo Mena": "José Heriberto Izquierdo Mena",
    "N'Golo KantÃ©": "N'Golo Kanté",
    "Adama TraorÃ©": "Adama Traoré",
    "Muhamed BeÅ¡iÄ\x87": "Muhamed Bešić"
}

merged_qw_2019_20['name'] = merged_qw_2019_20['name'].replace(players_names_mapping)

Check the differences again after the correction.

In [141]:
names_2019_2020 = set(player_teams_2019_2020['name'].unique())
names_merged = set(merged_qw_2019_20['name'].unique())

diff_2019_2020_and_merged = names_2019_2020 - names_merged

diff_merged_and_2019_2020 = names_merged - names_2019_2020

print('Names unique to player_teams_2019_2020:', diff_2019_2020_and_merged)
print('Names unique to merged_qw_2019_20:', diff_merged_and_2019_2020)

Names unique to player_teams_2019_2020: set()
Names unique to merged_qw_2019_20: set()


In [142]:
players_performances_2019_2020 = merged_qw_2019_20.merge(player_teams_2019_2020, how='left', on='name')

In [143]:
players_performances_2019_2020['Date'] = pd.to_datetime(players_performances_2019_2020.loc[:, 'kickoff_time']).dt.strftime('%d/%m/%y')

In [144]:
players_performances_2019_2020

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,team,Date
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,0,0,0,0,50,True,0,1,West Ham,10/08/19
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,1,0,0,0,50,True,0,1,Burnley,10/08/19
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,0,0,0,0,50,False,0,1,Brighton,10/08/19
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,2,0,0,0,45,True,0,1,Bournemouth,10/08/19
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,8,0,0,0,55,True,0,1,Man Utd,11/08/19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,2,-3103,1905,5008,61,True,0,47,Leicester,26/07/20
22556,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,7,77,147,70,50,False,0,47,Brighton,26/07/20
22557,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,0,153,325,172,40,True,0,47,Arsenal,26/07/20
22558,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,0,-95515,165,95680,49,True,0,47,Leicester,26/07/20


In [145]:
players_performances_2019_2020['opponent_team'].isna().sum()

0

Merge the dataframes on the `opponent_team` column.

In [146]:
teams_2019_2020 = pd.read_csv('data/2019_20/teams.csv')

teams_2019_2020 = teams_2019_2020[['id', 'name']]

players_performances_2019_2020 = pd.merge(players_performances_2019_2020,
                                         teams_2019_2020,
                                         left_on='opponent_team',
                                         right_on='id',
                                         how='left')
players_performances_2019_2020 = players_performances_2019_2020.rename(columns={'name_y': 'opp_team_name', 'name_x': 'name'})

players_performances_2019_2020 = players_performances_2019_2020.drop(columns='id')


In [147]:
players_performances_2019_2020

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,team,Date,opp_team_name
0,Aaron Cresswell,0,0,7,0,1.5,376,8,5,0,...,0,0,0,50,True,0,1,West Ham,10/08/19,Man City
1,Aaron Lennon,0,0,3,0,0.0,430,3,0,0,...,0,0,0,50,True,0,1,Burnley,10/08/19,Southampton
2,Aaron Mooy,0,0,0,0,0.0,516,7,0,0,...,0,0,0,50,False,0,1,Brighton,10/08/19,Watford
3,Aaron Ramsdale,0,0,11,0,0.0,494,2,1,0,...,0,0,0,45,True,0,1,Bournemouth,10/08/19,Sheffield Utd
4,Aaron Wan-Bissaka,0,2,34,1,16.1,122,9,0,0,...,0,0,0,55,True,0,1,Man Utd,11/08/19,Chelsea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Youri Tielemans,0,0,11,0,37.4,448,376,1,0,...,-3103,1905,5008,61,True,0,47,Leicester,26/07/20,Man Utd
22556,Yves Bissouma,0,0,22,0,1.3,53,372,1,1,...,77,147,70,50,False,0,47,Brighton,26/07/20,Burnley
22557,Zech Medley,0,0,0,0,0.0,636,371,0,0,...,153,325,172,40,True,0,47,Arsenal,26/07/20,Watford
22558,Çaglar Söyüncü,0,0,0,0,0.0,164,376,0,0,...,-95515,165,95680,49,True,0,47,Leicester,26/07/20,Man Utd



Add season attribute with the value `2019-20`.

In [148]:
players_performances_2019_2020['season_x'] = '2019-20'

In [149]:
players_performances_2019_2020.to_csv('data/2019_20/players_performances_2019_20.csv', index=False)

In [150]:
players_performances_2019_2020_with_team = pd.read_csv('data/2019_20/players_performances_2019_20.csv')

In [151]:
players_performances_2019_2020_with_team['opp_team_name'].isna().sum()

0

## 2020-21 season
In this season we have the team names in the `team_x` column so we can directly use it. So i only check if there are missing values in the `team_x` column and rename the column to `team`. Then save the data to a csv file.

In [152]:
players_performances_2020_2021 = merged_players_performances[merged_players_performances['season_x'] == '2020-21']

matches_2020_2021 = check_and_convert(pd.read_csv('data/2020_21/matches_2020_21.csv'))

players_performances_2020_2021

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
29693,2020-21,Aaron Cresswell,1,2,33,1,7.1,435,163,0,...,67591,41046,53,False,0,17,DEF,West Ham,Everton,01/01/21
29563,2020-21,Aaron Wan-Bissaka,1,1,31,0,26.2,313,164,1,...,27750,16420,53,True,0,17,DEF,Man Utd,Aston Villa,01/01/21
29333,2020-21,Abdoulaye Doucouré,0,0,16,0,11.6,512,163,1,...,3133,3665,53,True,0,17,MID,Everton,West Ham,01/01/21
29537,2020-21,Ahmed El Mohamady,0,0,0,0,0.0,29,164,0,...,1357,2420,43,False,0,17,DEF,Aston Villa,Man United,01/01/21
29389,2020-21,Albian Ajeti,0,0,0,0,0.0,447,163,0,...,29,21,55,False,0,17,FWD,West Ham,Everton,01/01/21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23701,2020-21,Wes Foderingham,0,0,0,0,0.0,350,66,0,...,161,343,44,True,0,7,GK,Sheffield Utd,Man City,31/10/20
23819,2020-21,Will Norris,0,0,0,0,0.0,583,60,0,...,1620,259,40,True,0,7,GK,Burnley,Chelsea,31/10/20
23550,2020-21,Willy Caballero,0,0,0,0,0.0,101,60,0,...,233,1200,48,False,0,7,GK,Chelsea,Burnley,31/10/20
23621,2020-21,Xherdan Shaqiri,1,0,17,0,15.7,247,63,0,...,187,219,64,True,0,7,MID,Liverpool,West Ham,31/10/20


In [153]:
players_performances_2020_2021['team_x'].isna().sum()

0

In [154]:
players_performances_2020_2021_with_team = players_performances_2020_2021.rename(columns={'team_x': 'team'})

In [155]:
players_performances_2020_2021_with_team['team'].isna().sum()

0

In [156]:
players_performances_2020_2021_with_team.to_csv('data/2020_21/players_performances_2020_21.csv', index=False)

## 2021-22 season
In this season we have the team names in the `team_x` column so we can directly use it. So i only check if there are missing values in the `team_x` column and rename the column to `team`. Then save the data to a csv file.

In [157]:
players_performances_2021_2022 = merged_players_performances[merged_players_performances['season_x'] == '2021-22']

matches_2021_2022 = check_and_convert(pd.read_csv('data/2021_22/matches_2021_22.csv'))

players_performances_2021_2022

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
56277,2021-22,Aaron Cresswell,0,0,0,0,0.0,411,204,0,...,1424,43658,53,False,0,21,DEF,West Ham,Crystal Palace,01/01/22
55949,2021-22,Aaron Ramsdale,0,0,9,0,0.0,559,201,2,...,52883,78741,51,True,0,21,GK,Arsenal,Man City,01/01/22
55883,2021-22,Adam Masina,0,0,10,0,0.8,390,210,1,...,195,136,43,True,0,21,DEF,Watford,Tottenham,01/01/22
56098,2021-22,Adedapo Awokoya-Mebude,0,0,0,0,0.0,469,210,0,...,0,113,45,True,0,21,FWD,Watford,Tottenham,01/01/22
55902,2021-22,Ainsley Maitland-Niles,0,0,0,0,0.0,13,201,0,...,28,215,52,True,0,21,MID,Arsenal,Man City,01/01/22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49814,2021-22,Tyrone Mings,0,0,2,0,0.8,38,91,2,...,7946,63492,50,True,0,10,DEF,Aston Villa,West Ham,31/10/21
50025,2021-22,Viljami Sinisalo,0,0,0,0,0.0,596,91,0,...,320,349,40,True,0,10,GK,Aston Villa,West Ham,31/10/21
50097,2021-22,Vladimir Coufal,0,0,3,0,0.6,418,91,0,...,2128,47841,48,False,0,10,DEF,West Ham,Aston Villa,31/10/21
50322,2021-22,Wesley Moraes,0,0,0,0,0.0,47,91,0,...,0,459,54,True,0,10,FWD,Aston Villa,West Ham,31/10/21


In [158]:
players_performances_2021_2022['team_x'].isna().sum()

0

In [159]:
players_performances_2021_2022_with_team = players_performances_2021_2022.rename(columns={'team_x': 'team'})

In [160]:
players_performances_2021_2022_with_team['team'].isna().sum()

0

In [161]:
players_performances_2021_2022_with_team.to_csv('data/2021_22/players_performances_2021_22.csv', index=False)

## 2022-23 season
In this season we have the team names in the `team_x` column so we can directly use it. So i only check if there are missing values in the `team_x` column and rename the column to `team`. Then save the data to a csv file.

In [162]:
players_performances_2022_2023 = merged_players_performances[merged_players_performances['season_x'] == '2022-23']

matches_2022_2023 = check_and_convert(pd.read_csv('data/2022_23/matches_2022_23.csv'))

players_performances_2022_2023

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
79943,2022-23,Alex Mighten,0,0,0,0,0.0,395,177,0,...,0,8,49,True,0,18,MID,Nott'm Forest,Chelsea,01/01/23
79914,2022-23,Alfie Devine,0,0,0,0,0.0,452,178,0,...,279,204,43,True,0,18,MID,Spurs,Aston Villa,01/01/23
80212,2022-23,Anwar El Ghazi,0,0,0,0,0.0,51,178,0,...,0,38,48,False,0,18,MID,Aston Villa,Tottenham,01/01/23
79681,2022-23,Armando Broja,0,0,0,0,0.0,150,177,0,...,100,4981,53,False,0,18,FWD,Chelsea,Nott'm Forest,01/01/23
80354,2022-23,Ashley Young,0,1,24,1,0.9,538,178,0,...,1295,4183,44,False,0,18,DEF,Aston Villa,Tottenham,01/01/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80073,2022-23,Yan Valery,0,0,0,0,0.0,414,173,0,...,0,514,40,False,0,18,DEF,Southampton,Fulham,31/12/22
80263,2022-23,Yerry Mina,0,0,0,0,0.0,188,175,0,...,7601,1294,43,False,0,18,DEF,Everton,Man City,31/12/22
79725,2022-23,Yerson Mosquera Valdelamar,0,0,0,0,0.0,494,180,0,...,1521,2283,39,True,0,18,DEF,Wolves,Man United,31/12/22
80030,2022-23,Zack Steffen,0,0,0,0,0.0,310,175,0,...,0,4,40,True,0,18,GK,Man City,Everton,31/12/22


In [163]:
players_performances_2022_2023['team_x'].isna().sum()

0

In [164]:
players_performances_2022_2023_with_team = players_performances_2022_2023.rename(columns={'team_x': 'team'})

In [165]:
players_performances_2022_2023_with_team['team'].isna().sum()

0

In [166]:
players_performances_2022_2023_with_team.to_csv('data/2022_23/players_performances_2022_23.csv', index=False)

## 2023-24 season
In this season we have the team names in the `team_x` column so we can directly use it. So i only check if there are missing values in the `team_x` column and rename the column to `team`. Then save the data to a csv file.

In [167]:
players_performances_2023_2024 = merged_players_performances[merged_players_performances['season_x'] == '2023-24']

matches_2023_2024 = check_and_convert(pd.read_csv('data/2023_24/matches_2023_24.csv'))

players_performances_2023_2024

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,position,team_x,opp_team_name,Date
110551,2023-24,Adrián San Miguel del Castillo,0,0,0,0,0.0,289,194,0,...,101,600,39,True,0,20,GK,Liverpool,Newcastle,01/01/24
110098,2023-24,Alex Murphy,0,0,0,0,0.0,634,194,0,...,1432,1789,39,False,0,20,DEF,Newcastle,Liverpool,01/01/24
110567,2023-24,Alexander Isak,0,0,29,0,13.7,415,194,4,...,21644,103599,76,False,0,20,FWD,Newcastle,Liverpool,01/01/24
110327,2023-24,Alexis Mac Allister,0,0,2,0,4.1,304,194,1,...,102,5705,58,True,0,20,MID,Liverpool,Newcastle,01/01/24
110452,2023-24,Alisson Ramses Becker,0,0,14,0,0.0,291,194,2,...,59292,19450,56,True,0,20,GK,Liverpool,Newcastle,01/01/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110484,2023-24,Tyrese Francois,0,0,0,0,0.0,272,193,0,...,35,82,44,True,0,20,MID,Fulham,Arsenal,31/12/23
110213,2023-24,William Saliba,0,0,11,0,2.8,20,193,2,...,101584,59376,56,False,1,20,DEF,Arsenal,Fulham,31/12/23
110653,2023-24,Willian Borges da Silva,0,0,6,0,38.4,591,193,1,...,221,30171,53,True,0,20,MID,Fulham,Arsenal,31/12/23
110429,2023-24,Yago de Santiago Alonso,0,0,0,0,0.0,754,198,0,...,28,24,45,True,0,20,MID,Spurs,Bournemouth,31/12/23


In [168]:
players_performances_2023_2024['team_x'].isna().sum()

0

In [169]:
players_performances_2023_2024_with_team = players_performances_2023_2024.rename(columns={'team_x': 'team'})

In [170]:
players_performances_2023_2024_with_team['team'].isna().sum()

0

In [171]:
players_performances_2023_2024_with_team.to_csv('data/2023_24/players_performances_2023_24.csv', index=False)

## Names correction
In this part I will correct the player names in the injuries data. I will use the player names from the player performances data and find the matches between the player names in the injuries data and the player names in the player performances data. Then I will correct the player names in the injuries data.

In [172]:
injuries['player_name'].nunique()

945

In [173]:
injuries['season_injured'].unique()

array(['2020-2021', '2018-2019', '2019-2020', '2022-2023', '2021-2022',
       '2017-2018', '2016-2017'], dtype=object)

Merge the player performances data for all seasons.

In [174]:
players_performances = pd.concat([
    players_performances_2016_2017_with_team,
    players_performances_2017_2018_with_team,
    players_performances_2018_2019_with_team,
    players_performances_2019_2020_with_team,
    players_performances_2020_2021_with_team,
    players_performances_2021_2022_with_team,
    players_performances_2022_2023_with_team,
    players_performances_2023_2024_with_team
])

In [175]:
players_performances.shape

(170425, 62)

In [176]:
players_performances['season_x'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [177]:
players_performances['name'].nunique()

1922

Find the common players between the injuries and player performances data.

In [178]:
unique_players_injuries = injuries['player_name'].unique()
unique_players_performances = players_performances['name'].unique()

common_players = set(unique_players_injuries).intersection(set(unique_players_performances))

In [179]:
len(common_players)

667

Find the players that are in the injuries data but not in the player performances data.

In [180]:
unique_players_injuries = injuries['player_name'].unique()
unique_players_performances = players_performances['name'].unique()

players_not_in_performances = set(unique_players_injuries) - set(unique_players_performances)

In [181]:
len(players_not_in_performances)

278

In [182]:
players_not_in_performances

{'Aaron Tshibola',
 'Abdallah Sima',
 'Abel Hernández',
 'Adam Federici',
 'Adlène Guédioura',
 'Adnan Januzaj',
 'Adrián',
 'Ahmed Elmohamady',
 'Ahmed Hegazy',
 'Aiden McGeady',
 'Aji Alese',
 'Aleksandar Kolarov',
 'Alex Bruce',
 'Alexander Sörloth',
 'Alisson',
 'Allan',
 'Allan McGregor',
 'André Gomes',
 'Antonio Barragán',
 'Arouna Koné',
 'Bacary Sagna',
 'Badou Ndiaye',
 'Bartosz Kapustka',
 'Bastian Schweinsteiger',
 'Ben Watson',
 'Benik Afobe',
 'Bernard',
 'Bernardo',
 'Bernardo Espinosa',
 'Bernardo Silva',
 'Billy Jones',
 'Boaz Myhill',
 'Borja Bastón',
 'Brad Smith',
 'Brendan Galloway',
 'Bruno Fernandes',
 'Bruno Guimarães',
 'Bruno Jordão',
 'Bruno Martins Indi',
 'Bryan Gil',
 'Bryan Oviedo',
 'Caglar Söyüncü',
 'Callum Elder',
 'Cameron Borthwick-Jackson',
 'Caoimhín Kelleher',
 'Carlos Vinícius',
 'Chicharito',
 'Chiquinho',
 'Chris Brunt',
 'Christian Nörgaard',
 'Chuba Akpom',
 'Chung-yong Lee',
 'Ché Adams',
 'Connor Goldson',
 'Costel Pantilimon',
 'Cristiano

For finding similarity between the player names in the injuries data and the player names in the player performances data I will use the `fuzzywuzzy` library. It is use for having one format of the player names in the injuries data and the player names in the player performances data. The similarity mean if there is something like this `Jean Michaël Seri` is same as `Jean Michael Seri` but the `é` is different from `e` and merge will not work.

In [183]:
unique_players_injuries = injuries['player_name'].unique()
unique_players_performances = players_performances['name'].unique()

In [184]:
names_matches = {}

for player_inj in unique_players_injuries:
    for player_perf in unique_players_performances:

        score = fuzz.ratio(player_inj.lower(), player_perf.lower())
        
        if 80 <= score < 100:
            if player_inj not in names_matches:
                names_matches[player_inj] = [(player_perf, score)]
            else:
                names_matches[player_inj].append((player_perf, score))

In [185]:
names_matches

{'Asmir Begovic': [('Asmir Begović', 92)],
 'Ola Aina': [('Olu Aina', 88)],
 'Christian Atsu': [('Christian Walton', 80)],
 'Harry Winks': [('Harry Lewis', 82)],
 'Dele Alli': [('Bamidele Alli', 82)],
 "Georges-Kevin N'Koudou": [('Georges-Kévin Nkoudou', 93)],
 'Emiliano Martínez': [('Emiliano Martínez Romero', 83),
  ('Emiliano Marcondes', 80),
  ('Lisandro Martínez', 82)],
 'Calum Chambers': [('Luke Chambers', 81)],
 'Aaron Ramsey': [('Aaron Ramsdale', 85)],
 'Dean Henderson': [('Jordan Henderson', 87)],
 'Joe Gomez': [('Joseph Gomez', 86)],
 'Jordan Henderson': [('Dean Henderson', 87)],
 'Philippe Coutinho': [('Philippe Coutinho Correia', 81)],
 'Muhamed Besic': [('Muhamed Bešić', 85)],
 'James McCarthy': [('James McArthur', 86), ('Alex McCarthy', 81)],
 'Havard Nordtveit': [('Håvard Nordtveit', 94)],
 'Alex McCarthy': [('James McCarthy', 81)],
 'Cédric Soares': [('Cédric Alves Soares', 81)],
 'Pierre-Emile Höjbjerg': [('Pierre-Emile Højbjerg', 95)],
 'Charlie Austin': [('Charlie Pa

Now I create the players mapping dictionary and remove the not matching names.

In [186]:
players = {
       'Asmir Begovic': [('Asmir Begović', 92)],
       'Ola Aina': [('Olu Aina', 88)],
       "Georges-Kevin N'Koudou": [('Georges-Kévin Nkoudou', 93)],
       'Emiliano Martínez': [('Emiliano Martínez Romero', 83),],
       'Joe Gomez': [('Joseph Gomez', 86)],
       'Philippe Coutinho': [('Philippe Coutinho Correia', 81)],
       'Muhamed Besic': [('Muhamed Bešić', 85)],
       'Havard Nordtveit': [('Håvard Nordtveit', 94)],
       'Cédric Soares': [('Cédric Alves Soares', 81)],
       'Pierre-Emile Höjbjerg': [('Pierre-Emile Højbjerg', 95)],
       'Jay Rodríguez': [('Jay Rodriguez', 92)],
       'Julián Speroni': [('Julian Speroni', 93)],
       'Ben Chilwell': [('Benjamin Chilwell', 83)],
       'Danny Drinkwater': [('Daniel Drinkwater', 85)],
       'Younès Kaboul': [('Younes Kaboul', 92)],
       'Jose Cholevas': [('José Holebas', 80)],
       'Étienne Capoue': [('Etienne Capoue', 93)],
       'Isaac Success': [('Isaac Success Ajayi', 81)],
       'Ahmed Elmohamady': [('Ahmed El Mohamady', 97)],
       'Javier Manquillo': [('Javier Manquillo Gaitán', 82)],
       'Jóhann Berg Gudmundsson': [('Johann Berg Gudmundsson', 96)],
       'Tom Dele-Bashiru': [('Ayotomiwa Dele-Bashiru', 84)],
       'Joe Willock': [('Joseph Willock', 88)],
       'Eddie Nketiah': [('Edward Nketiah', 81)],
       'Alexander Sörloth': [('Alexander Sørloth', 94)],
       'Martin Dúbravka': [('Martin Dubravka', 93)],
       'Rob Elliot': [('Robert Elliot', 87)],
       'Freddie Woodman': [('Frederick Woodman', 81)],
       'Aleksandar Mitrovic': [('Aleksandar Mitrović', 95)],
       'Steve Mounié': [('Steve Mounie', 92)],
       'Eric García': [('Eric Garcia', 91)],
       'Caoimhín Kelleher': [('Caoimhin Kelleher', 94)],
       'Naby Keïta': [('Naby Keita', 90)],
       'Mattéo Guendouzi': [('Matteo Guendouzi', 94)],
       'Emile Smith Rowe': [('Emile Smith-Rowe', 94)],
       'Caglar Söyüncü': [('Çaglar Söyüncü', 93)],
       'Jefferson Lerma': [('Jefferson Lerma Solís', 83)],
       'Ryan Bennett': [('Rhys Bennett', 83)],
       'Steven Sessegnon': [('Zeze Steven Sessegnon', 86)],
       'Jean Michaël Seri': [('Jean Michael Seri', 94)],
       'Aboubakar Kamara': [('Boubacar Kamara', 90)],
       'Kell Watts': [('Kelland Watts', 87)],
       'Miguel Almirón': [('Miguel Almirón Rejala', 80)],
       'Bobby De Cordova-Reid': [('Bobby Decordova-Reid', 98)],
       'Tanguy Ndombélé': [('Tanguy Ndombele', 87), ('Tanguy Ndombélé Alvaro', 81)],
       'Bruno Fernandes': [('Bruno Borges Fernandes', 81)],
       'Gabriel Martinelli': [('Gabriel Martinelli Silva', 86)],
       'David Martin': [('David Raya Martin', 83)],
       'Matty Longstaff': [('Matthew Longstaff', 88)],
       'Örjan Nyland': [('Ørjan Nyland', 92)],
       'Matija Sarkic': [('Matija Šarkić', 85)],
       'Björn Engels': [('Bjorn Engels', 92)],
       'Will Smallbone': [('William Smallbone', 90)],
       'Ché Adams': [('Che Adams', 89)],
       'Ismaïla Sarr': [('Ismaila Sarr', 92)],
       'Ralf Fährmann': [('Ralf Fahrmann', 92)],
       'Emiliano Buendía': [('Emiliano Buendía Stati', 84)],
       'Tino Livramento': [('Valentino Livramento', 86)],
       'Amad Diallo': [('Amadou Diallo', 92)],
        'Facundo Pellistri': [('Facundo Pellistri Rebollo', 81)],
       'Rayan Aït-Nouri': [('Rayan Ait Nouri', 87)],
       'Fábio Silva': [('Fabio Silva', 91)],
       'Matty Cash': [('Matthew Cash', 82)],
       'Aji Alese': [('Ajibola Alese', 82)],
       'Moisés Caicedo': [('Moisés Caicedo Corozo', 80)],
       'Marek Rodak': [('Marek Rodák', 91)],
       'Fábio Carvalho': [('Fabio Carvalho', 93)],
       'Roméo Lavia': [('Romeo Lavia', 91)],
       'Alejandro Garnacho': [('Alejandro Garnacho Ferreyra', 80)],
       'Vitaliy Mykolenko': [('Vitalii Mykolenko', 94)],
       'Jeremy Sarmiento': [('Jeremy Sarmiento Morante', 80)],
       'Mads Bech Sörensen': [('Mads Bech Sørensen', 94)],
       'Christian Nörgaard': [('Christian Nørgaard', 94)],
 }

For each record in the merged players performance data I will replace the player names with the correct names.

In [187]:
for player, matches in players.items():
    for match in matches:
        players_performances['name'] = players_performances['name'].replace(match[0], player)

In [188]:
players_performances['name'].nunique()

1884

Check the differences again after the correction.

In [189]:
unique_players_injuries_test = injuries['player_name'].unique()
unique_players_performances_test = players_performances['name'].unique()
names_matches_80_test = {}

for player_inj in unique_players_injuries_test:
    for player_perf in unique_players_performances_test:

        score = fuzz.ratio(player_inj.lower(), player_perf.lower())

        if score >= 80 and player_inj != player_perf:
            if player_inj not in names_matches_80_test:
                names_matches_80_test[player_inj] = [(player_perf, score)]
            else:
                names_matches_80_test[player_inj].append((player_perf, score))

In the `names_matches_80_test` we have only the not same names. Only one name is not corrected. So I will correct it manually.

In [190]:
names_matches_80_test

{'Christian Atsu': [('Christian Walton', 80)],
 'Harry Winks': [('Harry Lewis', 82)],
 'Dele Alli': [('Bamidele Alli', 82)],
 'Heung-min Son': [('Heung-Min Son', 100)],
 'Emiliano Martínez': [('Emiliano Marcondes', 80), ('Lisandro Martínez', 82)],
 'Calum Chambers': [('Luke Chambers', 81)],
 'Aaron Ramsey': [('Aaron Ramsdale', 85)],
 'Dean Henderson': [('Jordan Henderson', 87)],
 'Jordan Henderson': [('Dean Henderson', 87)],
 'James McCarthy': [('James McArthur', 86), ('Alex McCarthy', 81)],
 'Alex McCarthy': [('James McCarthy', 81)],
 'Charlie Austin': [('Charlie Patino', 86)],
 'Jay Rodríguez': [('James Rodríguez', 86)],
 'James McArthur': [('James McCarthy', 86)],
 'Glen Johnson': [('Ben Johnson', 87)],
 'Oliver McBurnie': [('Oliver Burke', 81)],
 'Andrew Robertson': [('Alex Robertson', 80)],
 'Abel Hernández': [('Onel Hernández', 86)],
 'Aaron Ramsdale': [('Aaron Ramsey', 85)],
 'Brad Smith': [('Adam Smith', 80)],
 'Harry Arter': [('Harry Tyrer', 82)],
 'Andrew Surman': [('Andrew M

In [191]:
players_performances.loc[players_performances['name'] == 'Heung-Min Son', 'name'] = 'Heung-min Son'

In [192]:
players_performances.shape

(170425, 62)

In [193]:
players_performances.to_csv('data/cleaned/cleaned_players_performances.csv', index=False)

In [194]:
players_performances = pd.read_csv('data/cleaned/cleaned_players_performances.csv', dtype={34: str, 37: str, 51: str})

# Matches data preparation

In [195]:
matches_2016_2017['season'] = '2016-17'
matches_2017_2018['season'] = '2017-18'
matches_2018_2019['season'] = '2018-19'
matches_2019_2020['season'] = '2019-20'
matches_2020_2021['season'] = '2020-21'
matches_2021_2022['season'] = '2021-22'
matches_2022_2023['season'] = '2022-23'
matches_2023_2024['season'] = '2023-24'

merged_matches = pd.concat(
    [matches_2016_2017, matches_2017_2018, matches_2018_2019, matches_2019_2020, matches_2020_2021, matches_2021_2022,
     matches_2022_2023, matches_2023_2024])


In [196]:
merged_matches.shape

(2938, 128)

In [197]:
merged_matches['season'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [198]:
players_performances['season_x'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [199]:
merged_matches['HomeTeam'].isna().sum()

0

In [200]:
merged_matches['AwayTeam'].isna().sum()

0

In [201]:
players_performances['team'].isna().sum()

0

Standardize the team names.

In [202]:
def standardize_format(array):
    array = array.str.lower()
    array = array.str.replace('spurs', 'tottenham')
    array = array.str.replace('man utd', 'man united')
    array = array.str.replace('sheffield utd', 'sheffield united')
    return array

merged_matches['HomeTeam'] = standardize_format(merged_matches['HomeTeam'])
merged_matches['AwayTeam'] = standardize_format(merged_matches['AwayTeam'])
players_performances['team'] = standardize_format(players_performances['team'])
players_performances['opp_team_name'] = standardize_format(players_performances['opp_team_name'])
injuries_df['club'] = standardize_format(injuries_df['club'])

In [203]:
merged_matches.to_csv('data/cleaned/cleaned_matches.csv', index=False)
players_performances.to_csv('data/cleaned/cleaned_players_performances.csv', index=False)
injuries_df.to_csv('data/cleaned/cleaned_injuries.csv', index=False)

In [204]:
merged_matches = pd.read_csv('data/cleaned/cleaned_matches.csv')

In [205]:
players_performances = pd.read_csv('data/cleaned/cleaned_players_performances.csv', dtype={34: str, 37: str, 51: str})

In [206]:
injuries_df = pd.read_csv('data/cleaned/cleaned_injuries.csv')

In [207]:
merged_matches.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,13/08/16,burnley,swansea,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,13/08/16,crystal palace,west brom,0,1,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,13/08/16,everton,tottenham,1,1,D,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,13/08/16,hull,leicester,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,13/08/16,man city,sunderland,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
matches_columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTR','HTHG','HTAG', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']

merged_matches = merged_matches[matches_columns]

### Renaming columns for better understanding
- HomeTeam - Home Team
- AwayTeam - Away Team
- Referee - Referee
- FT_HT_Goals - Full Time Home Team Goals
- FT_AT_Goals - Full Time Away Team Goals
- FT_Result - Full Time Result
- HT_Result - Half Time Result
- HT_HT_Goals - Half Time Home Team Goals
- HT_AT_Goals - Half Time Away Team Goals
- HT_Shots - Home Team Shots
- AT_Shots - Away Team Shots
- HT_Shots_target - Home Team Shots on Target
- AT_Shots_target - Away Team Shots on Target
- HT_Fouls_Comm - Home Team Fouls Committed
- AT_Fouls_Comm - Away Team Fouls Committed
- HT_Corners - Home Team Corners
- AT_Corners - Away Team Corners
- HT_Yellows - Home Team Yellow Cards
- AT_Yellows - Away Team Yellow Cards
- HT_Reds - Home Team Red Cards
- AT_Reds - Away Team Red Cards
- HT_W_Odds - Home Team Win Odds
- Draw_Odds - Draw Odds
- AT_W_Odds - Away Team Win Odds
- Date - Date
- Time - Time

In [209]:
merged_matches.columns = merged_matches.columns.str.replace(' ', '_').str.lower()
merged_matches.rename(columns={
    'hometeam': 'HomeTeam',
    'awayteam': 'AwayTeam',
    'referee': 'Referee',
    'fthg': 'FT_HT_Goals',
    'ftag': 'FT_AT_Goals',
    'ftr': 'FT_Result',
    'htr': 'HT_Result',
    'hthg': 'HT_HT_Goals',
    'htag': 'HT_AT_Goals',
    'hs': 'HT_Shots',
    'as': 'AT_Shots',
    'hst': 'HT_Shots_target',
    'ast': 'AT_Shots_target',
    'hf': 'HT_Fouls_Comm',
    'af': 'AT_Fouls_Comm',
    'hc': 'HT_Corners',
    'ac': 'AT_Corners',
    'hy': 'HT_Yellows',
    'ay': 'AT_Yellows',
    'hr': 'HT_Reds',
    'ar': 'AT_Reds',
    'b365h': 'HT_W_Odds',
    'b365d': 'Draw_Odds',
    'b365a': 'AT_W_Odds',
    'date': 'Date'
}, inplace=True)


In [210]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,AT_Fouls_Comm,HT_Corners,AT_Corners,HT_Yellows,AT_Yellows,HT_Reds,AT_Reds,HT_W_Odds,Draw_Odds,AT_W_Odds
0,13/08/16,burnley,swansea,0,1,A,D,0,0,J Moss,...,14,7,4,3,2,0,0,2.40,3.3,3.25
1,13/08/16,crystal palace,west brom,0,1,A,D,0,0,C Pawson,...,15,3,6,2,2,0,0,2.00,3.3,4.50
2,13/08/16,everton,tottenham,1,1,D,H,1,0,M Atkinson,...,14,5,6,0,0,0,0,3.20,3.4,2.40
3,13/08/16,hull,leicester,2,1,H,H,1,0,M Dean,...,17,5,3,2,2,0,0,4.50,3.6,1.91
4,13/08/16,man city,sunderland,2,1,H,H,1,0,R Madley,...,14,9,6,1,2,0,0,1.25,6.5,15.00


In [211]:
def calculate_rolling_average(dataframe, columns):
    dataframe.sort_values('Date', inplace=True)
    for col in columns:
        # Create a dictionary with empty deque for each team
        teams = {team: deque(maxlen=5) for team in
                      set(dataframe['HomeTeam'].unique()).union(dataframe['AwayTeam'].unique())}\
        
        # Create new columns that will store the last 5 games results for home and away teams
        dataframe['5GAV_' + col[0]] = None
        dataframe['5GAV_' + col[1]] = None
        
        # Cycle through the dataframe in order to calculate the average of the last 5 games
        for index, row in dataframe.iterrows():
            # Calculate the average of the last 5 games
            last_5_games_home = np.sum(teams[row['HomeTeam']]) / 5
            last_5_games_away = np.sum(teams[row['AwayTeam']]) / 5

            # Update the Last_5_Games columns with the sums
            dataframe.at[index, '5GAV_' + col[0]] = last_5_games_home
            dataframe.at[index, '5GAV_' + col[1]] = last_5_games_away

            # Now update the team's games results with the current row
            teams[row['HomeTeam']].append(row[col[0]])
            teams[row['AwayTeam']].append(row[col[1]])

        dataframe['5GAV_' + col[0]] = dataframe['5GAV_' + col[0]].astype(float)
        dataframe['5GAV_' + col[1]] = dataframe['5GAV_' + col[1]].astype(float)

    return dataframe

In [212]:
merged_matches = merged_matches.sort_values(by=['HomeTeam'], ascending=[True])

In [213]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,AT_Fouls_Comm,HT_Corners,AT_Corners,HT_Yellows,AT_Yellows,HT_Reds,AT_Reds,HT_W_Odds,Draw_Odds,AT_W_Odds
2182,16/03/22,arsenal,liverpool,0,2,A,D,0,0,A Marriner,...,9,1,6,0,1,0,0,3.50,4.00,1.90
2364,09/10/22,arsenal,liverpool,3,2,H,H,2,1,M Oliver,...,11,4,3,1,2,0,0,2.50,3.60,2.62
1639,16/12/20,arsenal,southampton,1,1,D,A,0,1,P Tierney,...,13,2,7,1,3,1,0,2.25,3.50,3.10
1237,27/10/19,arsenal,crystal palace,2,2,D,H,2,1,M Atkinson,...,9,12,5,2,0,0,0,1.45,4.75,6.50
1980,22/10/21,arsenal,aston villa,3,1,H,H,2,0,C Pawson,...,10,7,4,2,5,0,0,1.83,3.75,4.20


In [214]:
# Columns to calculate the rolling average
calc_columns = [['FT_HT_Goals', 'FT_AT_Goals'], ['HT_Shots', 'AT_Shots'], ['HT_Shots_target', 'AT_Shots_target'],['HT_Fouls_Comm', 'AT_Fouls_Comm'],['HT_Corners', 'AT_Corners'],['HT_Yellows', 'AT_Yellows'],['HT_Reds', 'AT_Reds']]

merged_matches = calculate_rolling_average(merged_matches, calc_columns)
merged_matches = merged_matches.sort_values(by=['HomeTeam'], ascending=[True])

In [215]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Shots_target,5GAV_AT_Shots_target,5GAV_HT_Fouls_Comm,5GAV_AT_Fouls_Comm,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds
188,01/01/17,arsenal,crystal palace,2,0,H,H,1,0,A Marriner,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,10/09/16,arsenal,southampton,2,1,H,D,1,1,R Madley,...,2.6,3.8,9.8,7.4,4.8,7.6,1.8,0.8,0.2,0.0
140,10/12/16,arsenal,stoke,3,1,H,D,1,1,L Mason,...,2.8,3.4,10.0,9.0,5.6,6.6,2.0,0.6,0.2,0.0
240,11/02/17,arsenal,hull,2,0,H,H,1,0,M Clattenburg,...,4.4,3.6,11.2,10.0,5.4,4.8,1.8,1.4,0.2,0.0
2491,11/02/23,arsenal,brentford,1,1,D,D,0,0,P Bankes,...,5.0,5.2,12.6,8.8,5.0,4.4,2.0,1.8,0.2,0.0


In [216]:
merged_matches[(merged_matches['HomeTeam'] == 'leicester') |(merged_matches['AwayTeam'] == 'leicester')]

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Shots_target,5GAV_AT_Shots_target,5GAV_HT_Fouls_Comm,5GAV_AT_Fouls_Comm,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds
380,11/08/17,arsenal,leicester,4,3,H,D,2,2,M Dean,...,7.6,6.0,10.0,11.8,4.8,5.2,1.2,1.8,0.0,0.2
2179,13/03/22,arsenal,leicester,2,0,H,H,1,0,A Taylor,...,4.8,4.0,10.4,9.8,4.4,3.2,2.2,1.4,0.0,0.2
2291,13/08/22,arsenal,leicester,4,2,H,H,2,0,D England,...,5.2,4.6,7.2,13.2,4.2,4.2,0.8,2.2,0.0,0.0
1472,07/07/20,arsenal,leicester,1,1,D,H,1,0,C Kavanagh,...,3.4,3.8,11.6,9.6,4.8,5.6,1.8,1.6,0.0,0.0
1575,25/10/20,arsenal,leicester,0,1,A,D,0,0,C Pawson,...,5.8,5.6,11.8,12.4,8.4,4.8,1.6,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,20/02/22,wolves,leicester,2,1,H,D,1,1,C Pawson,...,2.4,3.4,12.4,10.2,3.4,6.8,1.8,1.0,0.2,0.0
1393,14/02/20,wolves,leicester,0,0,D,D,0,0,M Dean,...,1.4,4.8,7.4,10.6,5.2,7.4,1.6,1.6,0.2,0.0
1741,07/02/21,wolves,leicester,0,0,D,D,0,0,M Atkinson,...,3.0,3.0,12.6,7.4,4.0,4.4,1.6,1.4,0.0,0.2
2393,23/10/22,wolves,leicester,0,4,A,A,0,2,M Oliver,...,3.8,3.6,14.2,11.0,4.4,4.8,3.2,1.8,0.2,0.2


In [217]:
merged_matches.head(10)

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Shots_target,5GAV_AT_Shots_target,5GAV_HT_Fouls_Comm,5GAV_AT_Fouls_Comm,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds
188,01/01/17,arsenal,crystal palace,2,0,H,H,1,0,A Marriner,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,10/09/16,arsenal,southampton,2,1,H,D,1,1,R Madley,...,2.6,3.8,9.8,7.4,4.8,7.6,1.8,0.8,0.2,0.0
140,10/12/16,arsenal,stoke,3,1,H,D,1,1,L Mason,...,2.8,3.4,10.0,9.0,5.6,6.6,2.0,0.6,0.2,0.0
240,11/02/17,arsenal,hull,2,0,H,H,1,0,M Clattenburg,...,4.4,3.6,11.2,10.0,5.4,4.8,1.8,1.4,0.2,0.0
2491,11/02/23,arsenal,brentford,1,1,D,D,0,0,P Bankes,...,5.0,5.2,12.6,8.8,5.0,4.4,2.0,1.8,0.2,0.0
677,11/03/18,arsenal,watford,3,0,H,H,1,0,M Atkinson,...,7.2,4.0,12.8,13.4,5.0,3.4,1.6,1.0,0.2,0.0
380,11/08/17,arsenal,leicester,4,3,H,D,2,2,M Dean,...,7.6,6.0,10.0,11.8,4.8,5.2,1.2,1.8,0.0,0.2
1931,11/09/21,arsenal,norwich,1,0,H,D,0,0,M Oliver,...,7.2,3.8,9.0,8.2,5.2,2.8,1.2,0.4,0.0,0.0
876,11/11/18,arsenal,wolves,1,1,D,A,0,1,S Attwell,...,6.2,3.2,8.6,10.0,5.6,4.4,1.4,1.4,0.0,0.2
2771,11/11/23,arsenal,burnley,3,1,H,H,1,0,M Oliver,...,5.4,3.2,8.0,8.8,7.0,6.4,1.4,1.6,0.0,0.2


In [218]:
merged_matches['Date'] = pd.to_datetime(merged_matches['Date'], format='%d/%m/%y')

merged_matches.sort_values(by='Date', inplace=True)

merged_matches.reset_index(drop=True, inplace=True)

In [219]:
merged_matches.head(10)

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Shots_target,5GAV_AT_Shots_target,5GAV_HT_Fouls_Comm,5GAV_AT_Fouls_Comm,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds
0,2016-08-13,middlesbrough,stoke,1,1,D,H,1,0,K Friend,...,2.6,3.4,14.2,10.4,4.0,4.2,2.2,1.0,0.0,0.0
1,2016-08-13,burnley,swansea,0,1,A,D,0,0,J Moss,...,4.8,4.6,9.0,9.6,5.0,4.4,1.4,1.4,0.0,0.0
2,2016-08-13,hull,leicester,2,1,H,H,1,0,M Dean,...,4.2,4.8,10.0,11.6,5.2,4.0,1.6,2.0,0.2,0.0
3,2016-08-13,crystal palace,west brom,0,1,A,D,0,0,C Pawson,...,3.8,3.0,12.4,12.2,5.6,4.8,1.0,2.0,0.0,0.0
4,2016-08-13,man city,sunderland,2,1,H,H,1,0,R Madley,...,5.0,2.6,11.6,11.6,6.8,5.4,1.6,1.8,0.0,0.2
5,2016-08-13,everton,tottenham,1,1,D,H,1,0,M Atkinson,...,3.8,4.6,11.4,11.8,4.4,4.0,0.8,2.4,0.2,0.0
6,2016-08-13,southampton,watford,1,1,D,A,0,1,R East,...,3.8,3.6,12.8,13.2,3.0,4.4,1.8,0.6,0.0,0.2
7,2016-08-14,arsenal,liverpool,3,4,A,D,1,1,M Oliver,...,3.4,6.8,10.6,9.8,5.4,5.6,1.0,1.2,0.0,0.0
8,2016-08-14,bournemouth,man united,1,3,A,A,0,1,A Marriner,...,4.2,3.8,9.8,12.0,5.2,4.6,1.4,2.2,0.0,0.0
9,2016-08-15,chelsea,west ham,2,1,H,D,0,0,A Taylor,...,4.4,4.2,8.6,9.8,6.4,4.6,1.6,1.4,0.0,0.2


In [220]:
def last_5_games_result(dataframe):
    # Create a dictionary with empty deque for each team
    teams = {team: deque(maxlen=5) for team in
             set(dataframe['HomeTeam'].unique()).union(dataframe['AwayTeam'].unique())}
    # Create new columns that will store the last 5 games results for home and away teams
    dataframe['HT_Last_5_Games'] = ''
    dataframe['AT_Last_5_Games'] = ''
    # Sort dataframe by Date
    dataframe.sort_values('Date', inplace=True)

    for index, row in dataframe.iterrows():
        last_5_games_home_str = ''.join(teams[row['HomeTeam']])
        last_5_games_away_str = ''.join(teams[row['AwayTeam']])
        # Update the Last_5_Games columns with the strings
        dataframe.at[index, 'HT_Last_5_Games'] = last_5_games_home_str
        dataframe.at[index, 'AT_Last_5_Games'] = last_5_games_away_str
        # Now update the team's games results with the current row
        if row['FT_Result'] == 'H':
            teams[row['HomeTeam']].append('W')
            teams[row['AwayTeam']].append('L')
        elif row['FT_Result'] == 'A':
            teams[row['HomeTeam']].append('L')
            teams[row['AwayTeam']].append('W')
        else:  # Draw
            teams[row['HomeTeam']].append('D')
            teams[row['AwayTeam']].append('D')
    return dataframe


In [221]:
merged_matches = last_5_games_result(merged_matches)

In [222]:
merged_matches[(merged_matches['HomeTeam'] == 'arsenal') |(merged_matches['AwayTeam'] == 'arsenal')].sort_values(by='Date')

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Fouls_Comm,5GAV_AT_Fouls_Comm,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games
7,2016-08-14,arsenal,liverpool,3,4,A,D,1,1,M Oliver,...,10.6,9.8,5.4,5.6,1.0,1.2,0.0,0.0,,
15,2016-08-20,leicester,arsenal,0,0,D,D,0,0,M Clattenburg,...,11.4,8.0,6.2,5.0,1.4,0.8,0.0,0.0,L,L
25,2016-08-27,watford,arsenal,1,3,A,A,0,3,K Friend,...,9.4,11.4,4.0,6.2,2.4,2.0,0.0,0.0,DL,LD
31,2016-09-10,arsenal,southampton,2,1,H,D,1,1,R Madley,...,9.8,7.4,4.8,7.6,1.8,0.8,0.2,0.0,LDW,DLD
41,2016-09-17,hull,arsenal,1,4,A,A,0,1,R East,...,12.0,11.4,5.4,4.8,2.4,1.4,0.0,0.4,WWLD,LDWW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,2024-02-11,west ham,arsenal,0,6,A,A,0,4,C Pawson,...,8.8,12.4,5.6,5.2,1.6,1.6,0.0,0.2,WDDDL,LLWWW
2903,2024-02-17,burnley,arsenal,0,5,A,A,0,2,J Gillett,...,8.8,11.0,4.2,5.0,1.0,1.8,0.0,0.2,LDLDL,LWWWW
2911,2024-02-24,arsenal,newcastle,4,1,H,H,2,0,P Tierney,...,10.0,8.4,5.2,5.0,2.6,2.4,0.0,0.0,WWWWW,LWDWD
2927,2024-03-04,sheffield united,arsenal,0,6,A,A,0,5,S Barrott,...,11.2,10.4,4.0,7.6,2.0,2.4,0.4,0.0,LLWLL,WWWWW


In [223]:
letter_mapping = {'W': 3, 'D': 1, 'L': 0}

def calculate_average_form(row):
    return sum(letter_mapping[letter] for letter in row)

merged_matches['HT_Form'] = merged_matches['HT_Last_5_Games'].apply(calculate_average_form)
merged_matches['AT_Form'] = merged_matches['AT_Last_5_Games'].apply(calculate_average_form)

In [224]:
merged_matches.shape

(2938, 43)

In [225]:
merged_matches.sort_values(by=['HomeTeam', 'Date'], ascending=[True,True])

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games,HT_Form,AT_Form
7,2016-08-14,arsenal,liverpool,3,4,A,D,1,1,M Oliver,...,5.4,5.6,1.0,1.2,0.0,0.0,,,0,0
31,2016-09-10,arsenal,southampton,2,1,H,D,1,1,R Madley,...,4.8,7.6,1.8,0.8,0.2,0.0,LDW,DLD,4,2
57,2016-09-24,arsenal,chelsea,3,0,H,H,3,0,M Oliver,...,5.8,4.4,1.2,1.4,0.0,0.0,LDWWW,WWWDL,10,10
73,2016-10-15,arsenal,swansea,3,2,H,H,2,1,J Moss,...,3.2,5.4,1.2,0.8,0.0,0.0,WWWWW,LDLLL,15,1
82,2016-10-22,arsenal,middlesbrough,0,0,D,D,0,0,M Dean,...,10.0,3.2,2.4,1.6,0.2,0.2,WWWWW,LLLDL,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,2023-12-30,wolves,everton,3,0,H,H,1,0,T Bramall,...,5.0,6.0,2.0,1.8,0.0,0.0,WDLWW,WWWLL,10,9
2877,2024-02-01,wolves,man united,3,4,A,A,0,2,J Gillett,...,2.6,5.2,0.6,1.6,0.0,0.0,LWWWD,DLWLD,10,5
2892,2024-02-10,wolves,brentford,0,2,A,A,0,1,S Hooper,...,6.2,4.4,1.8,1.2,0.0,0.0,WWDLW,LLWLL,10,3
2916,2024-02-25,wolves,sheffield united,1,0,H,H,1,0,D Bond,...,6.0,4.6,1.8,1.6,0.0,0.0,DLWLW,DLLWL,7,4


In [226]:
merged_matches.shape

(2938, 43)

In [227]:
# Specify columns to use for removing first 5 games
# This is important as to not use values obtained in the first 5 games of the season
# This could affect the models scores

cols = ['HT_Last_5_Games', 'AT_Last_5_Games']

# Drop first 5 games of the season for each team
merged_matches = merged_matches[
    merged_matches[cols].apply(lambda x: x.map(lambda y: len(y) == 5)).all(axis=1)]

In [228]:
merged_matches.shape

(2824, 43)

In [229]:
merged_matches[(merged_matches['HomeTeam'] == 'arsenal') |(merged_matches['AwayTeam'] == 'arsenal')].sort_values(by='Date')

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games,HT_Form,AT_Form
57,2016-09-24,arsenal,chelsea,3,0,H,H,3,0,M Oliver,...,5.8,4.4,1.2,1.4,0.0,0.0,LDWWW,WWWDL,10,10
66,2016-10-02,burnley,arsenal,0,1,A,D,0,0,C Pawson,...,4.6,6.0,1.8,2.0,0.0,0.2,WLDLW,DWWWW,7,13
73,2016-10-15,arsenal,swansea,3,2,H,H,2,1,J Moss,...,3.2,5.4,1.2,0.8,0.0,0.0,WWWWW,LDLLL,15,1
82,2016-10-22,arsenal,middlesbrough,0,0,D,D,0,0,M Dean,...,10.0,3.2,2.4,1.6,0.2,0.2,WWWWW,LLLDL,15,1
93,2016-10-29,sunderland,arsenal,1,4,A,A,0,1,M Atkinson,...,5.0,4.8,2.4,1.8,0.0,0.0,LLDLL,WWWWD,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,2024-02-11,west ham,arsenal,0,6,A,A,0,4,C Pawson,...,5.6,5.2,1.6,1.6,0.0,0.2,WDDDL,LLWWW,6,9
2903,2024-02-17,burnley,arsenal,0,5,A,A,0,2,J Gillett,...,4.2,5.0,1.0,1.8,0.0,0.2,LDLDL,LWWWW,2,12
2911,2024-02-24,arsenal,newcastle,4,1,H,H,2,0,P Tierney,...,5.2,5.0,2.6,2.4,0.0,0.0,WWWWW,LWDWD,15,8
2927,2024-03-04,sheffield united,arsenal,0,6,A,A,0,5,S Barrott,...,4.0,7.6,2.0,2.4,0.4,0.0,LLWLL,WWWWW,3,15


In [230]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games,HT_Form,AT_Form
57,2016-09-24,arsenal,chelsea,3,0,H,H,3,0,M Oliver,...,5.8,4.4,1.2,1.4,0.0,0.0,LDWWW,WWWDL,10,10
50,2016-09-24,man united,leicester,4,1,H,H,4,0,M Dean,...,6.4,6.0,1.6,0.8,0.0,0.2,WWWLL,LDWLW,9,7
51,2016-09-24,swansea,man city,1,3,A,D,1,1,N Swarbrick,...,4.8,4.4,1.2,1.2,0.0,0.0,WLLDL,WWWWW,4,15
52,2016-09-24,sunderland,crystal palace,2,3,A,H,1,0,A Taylor,...,3.4,4.6,1.8,2.0,0.2,0.0,LLDLL,LLDWW,1,7
53,2016-09-24,stoke,west brom,1,1,D,D,0,0,M Atkinson,...,3.4,5.0,2.2,1.4,0.0,0.0,DLLLL,WLDLW,1,7


In [231]:
merged_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2824 entries, 57 to 2937
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  2824 non-null   datetime64[ns]
 1   HomeTeam              2824 non-null   object        
 2   AwayTeam              2824 non-null   object        
 3   FT_HT_Goals           2824 non-null   int64         
 4   FT_AT_Goals           2824 non-null   int64         
 5   FT_Result             2824 non-null   object        
 6   HT_Result             2824 non-null   object        
 7   HT_HT_Goals           2824 non-null   int64         
 8   HT_AT_Goals           2824 non-null   int64         
 9   Referee               2824 non-null   object        
 10  HT_Shots              2824 non-null   int64         
 11  AT_Shots              2824 non-null   int64         
 12  HT_Shots_target       2824 non-null   int64         
 13  AT_Shots_target       

In [232]:
merged_matches.isnull().sum()

Date                    0
HomeTeam                0
AwayTeam                0
FT_HT_Goals             0
FT_AT_Goals             0
FT_Result               0
HT_Result               0
HT_HT_Goals             0
HT_AT_Goals             0
Referee                 0
HT_Shots                0
AT_Shots                0
HT_Shots_target         0
AT_Shots_target         0
HT_Fouls_Comm           0
AT_Fouls_Comm           0
HT_Corners              0
AT_Corners              0
HT_Yellows              0
AT_Yellows              0
HT_Reds                 0
AT_Reds                 0
HT_W_Odds               0
Draw_Odds               0
AT_W_Odds               0
5GAV_FT_HT_Goals        0
5GAV_FT_AT_Goals        0
5GAV_HT_Shots           0
5GAV_AT_Shots           0
5GAV_HT_Shots_target    0
5GAV_AT_Shots_target    0
5GAV_HT_Fouls_Comm      0
5GAV_AT_Fouls_Comm      0
5GAV_HT_Corners         0
5GAV_AT_Corners         0
5GAV_HT_Yellows         0
5GAV_AT_Yellows         0
5GAV_HT_Reds            0
5GAV_AT_Reds

In [233]:
def parse_date(date_obj):
    if isinstance(date_obj, pd.Timestamp):
        return date_obj.strftime('%d/%m/%y')
    else:
        return None  

merged_matches['Date'] = merged_matches['Date'].apply(parse_date)

In [234]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Corners,5GAV_AT_Corners,5GAV_HT_Yellows,5GAV_AT_Yellows,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games,HT_Form,AT_Form
57,24/09/16,arsenal,chelsea,3,0,H,H,3,0,M Oliver,...,5.8,4.4,1.2,1.4,0.0,0.0,LDWWW,WWWDL,10,10
50,24/09/16,man united,leicester,4,1,H,H,4,0,M Dean,...,6.4,6.0,1.6,0.8,0.0,0.2,WWWLL,LDWLW,9,7
51,24/09/16,swansea,man city,1,3,A,D,1,1,N Swarbrick,...,4.8,4.4,1.2,1.2,0.0,0.0,WLLDL,WWWWW,4,15
52,24/09/16,sunderland,crystal palace,2,3,A,H,1,0,A Taylor,...,3.4,4.6,1.8,2.0,0.2,0.0,LLDLL,LLDWW,1,7
53,24/09/16,stoke,west brom,1,1,D,D,0,0,M Atkinson,...,3.4,5.0,2.2,1.4,0.0,0.0,DLLLL,WLDLW,1,7


In [235]:
merged_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2824 entries, 57 to 2937
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  2824 non-null   object 
 1   HomeTeam              2824 non-null   object 
 2   AwayTeam              2824 non-null   object 
 3   FT_HT_Goals           2824 non-null   int64  
 4   FT_AT_Goals           2824 non-null   int64  
 5   FT_Result             2824 non-null   object 
 6   HT_Result             2824 non-null   object 
 7   HT_HT_Goals           2824 non-null   int64  
 8   HT_AT_Goals           2824 non-null   int64  
 9   Referee               2824 non-null   object 
 10  HT_Shots              2824 non-null   int64  
 11  AT_Shots              2824 non-null   int64  
 12  HT_Shots_target       2824 non-null   int64  
 13  AT_Shots_target       2824 non-null   int64  
 14  HT_Fouls_Comm         2824 non-null   int64  
 15  AT_Fouls_Comm         282

# Encoding

## Matches encoding

In [236]:
merged_matches['HomeTeam'].unique()

array(['arsenal', 'man united', 'swansea', 'sunderland', 'stoke',
       'bournemouth', 'liverpool', 'middlesbrough', 'west ham', 'burnley',
       'everton', 'watford', 'hull', 'tottenham', 'leicester',
       'crystal palace', 'west brom', 'chelsea', 'man city',
       'southampton', 'brighton', 'huddersfield', 'newcastle', 'cardiff',
       'fulham', 'wolves', 'aston villa', 'sheffield united', 'norwich',
       'leeds', 'brentford', "nott'm forest", 'luton'], dtype=object)

In [237]:
merged_matches['AwayTeam'].unique()

array(['chelsea', 'leicester', 'man city', 'crystal palace', 'west brom',
       'everton', 'hull', 'tottenham', 'southampton', 'watford',
       'middlesbrough', 'liverpool', 'bournemouth', 'stoke', 'arsenal',
       'west ham', 'sunderland', 'swansea', 'burnley', 'man united',
       'huddersfield', 'newcastle', 'brighton', 'wolves', 'fulham',
       'cardiff', 'sheffield united', 'norwich', 'aston villa', 'leeds',
       'brentford', "nott'm forest", 'luton'], dtype=object)

In [238]:
unique_teams = merged_matches['HomeTeam'].unique()
unique_teams_sorted = sorted(unique_teams)

team_to_id = {team: idx for idx, team in enumerate(unique_teams_sorted, start=1)}

In [239]:
team_to_id

{'arsenal': 1,
 'aston villa': 2,
 'bournemouth': 3,
 'brentford': 4,
 'brighton': 5,
 'burnley': 6,
 'cardiff': 7,
 'chelsea': 8,
 'crystal palace': 9,
 'everton': 10,
 'fulham': 11,
 'huddersfield': 12,
 'hull': 13,
 'leeds': 14,
 'leicester': 15,
 'liverpool': 16,
 'luton': 17,
 'man city': 18,
 'man united': 19,
 'middlesbrough': 20,
 'newcastle': 21,
 'norwich': 22,
 "nott'm forest": 23,
 'sheffield united': 24,
 'southampton': 25,
 'stoke': 26,
 'sunderland': 27,
 'swansea': 28,
 'tottenham': 29,
 'watford': 30,
 'west brom': 31,
 'west ham': 32,
 'wolves': 33}

In [240]:
team_to_id_df = pd.DataFrame(team_to_id.items(), columns=['team', 'team_id'])
team_to_id_df.to_csv('data/team_to_id.csv', index=False)

In [241]:
merged_matches['HT_Code'] = merged_matches['HomeTeam'].map(team_to_id)

merged_matches['AT_Code'] = merged_matches['AwayTeam'].map(team_to_id)


In [242]:
merged_matches.dtypes

Date                     object
HomeTeam                 object
AwayTeam                 object
FT_HT_Goals               int64
FT_AT_Goals               int64
FT_Result                object
HT_Result                object
HT_HT_Goals               int64
HT_AT_Goals               int64
Referee                  object
HT_Shots                  int64
AT_Shots                  int64
HT_Shots_target           int64
AT_Shots_target           int64
HT_Fouls_Comm             int64
AT_Fouls_Comm             int64
HT_Corners                int64
AT_Corners                int64
HT_Yellows                int64
AT_Yellows                int64
HT_Reds                   int64
AT_Reds                   int64
HT_W_Odds               float64
Draw_Odds               float64
AT_W_Odds               float64
5GAV_FT_HT_Goals        float64
5GAV_FT_AT_Goals        float64
5GAV_HT_Shots           float64
5GAV_AT_Shots           float64
5GAV_HT_Shots_target    float64
5GAV_AT_Shots_target    float64
5GAV_HT_

Predict the result, if the home team wins the result is 1, if the match is a draw or the away team wins the result is 2.

In [243]:
result_mapping = {'H': 1, 'D': 0, 'A': 0}

merged_matches['FT_Result_code'] = merged_matches['FT_Result'].map(result_mapping)
merged_matches['HT_Result_code'] = merged_matches['HT_Result'].map(result_mapping)

In [244]:
merged_matches.head()

,Date,HomeTeam,AwayTeam,FT_HT_Goals,FT_AT_Goals,FT_Result,HT_Result,HT_HT_Goals,HT_AT_Goals,Referee,...,5GAV_HT_Reds,5GAV_AT_Reds,HT_Last_5_Games,AT_Last_5_Games,HT_Form,AT_Form,HT_Code,AT_Code,FT_Result_code,HT_Result_code
57,24/09/16,arsenal,chelsea,3,0,H,H,3,0,M Oliver,...,0.0,0.0,LDWWW,WWWDL,10,10,1,8,1,1
50,24/09/16,man united,leicester,4,1,H,H,4,0,M Dean,...,0.0,0.2,WWWLL,LDWLW,9,7,19,15,1,1
51,24/09/16,swansea,man city,1,3,A,D,1,1,N Swarbrick,...,0.0,0.0,WLLDL,WWWWW,4,15,28,18,0,0
52,24/09/16,sunderland,crystal palace,2,3,A,H,1,0,A Taylor,...,0.2,0.0,LLDLL,LLDWW,1,7,27,9,0,1
53,24/09/16,stoke,west brom,1,1,D,D,0,0,M Atkinson,...,0.0,0.0,DLLLL,WLDLW,1,7,26,31,0,0


In [245]:
merged_matches = merged_matches.drop(columns=['HomeTeam', 'AwayTeam', 'FT_Result', 'HT_Result', 'Referee', 'HT_Last_5_Games', 'AT_Last_5_Games'])

In [246]:
merged_matches.dtypes

Date                     object
FT_HT_Goals               int64
FT_AT_Goals               int64
HT_HT_Goals               int64
HT_AT_Goals               int64
HT_Shots                  int64
AT_Shots                  int64
HT_Shots_target           int64
AT_Shots_target           int64
HT_Fouls_Comm             int64
AT_Fouls_Comm             int64
HT_Corners                int64
AT_Corners                int64
HT_Yellows                int64
AT_Yellows                int64
HT_Reds                   int64
AT_Reds                   int64
HT_W_Odds               float64
Draw_Odds               float64
AT_W_Odds               float64
5GAV_FT_HT_Goals        float64
5GAV_FT_AT_Goals        float64
5GAV_HT_Shots           float64
5GAV_AT_Shots           float64
5GAV_HT_Shots_target    float64
5GAV_AT_Shots_target    float64
5GAV_HT_Fouls_Comm      float64
5GAV_AT_Fouls_Comm      float64
5GAV_HT_Corners         float64
5GAV_AT_Corners         float64
5GAV_HT_Yellows         float64
5GAV_AT_

In [247]:
merged_matches.to_csv('data/encoded/encoded_matches.csv', index=False)

## Player performances encoding

In [248]:
cleaned_players_performances = pd.read_csv('data/cleaned/cleaned_players_performances.csv', dtype={34: str, 37: str, 51: str})

In [249]:
cleaned_players_performances.shape

(170425, 62)

In [250]:
cleaned_players_performances.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,loaned_in,loaned_out,offside,open_play_crosses,penalties_conceded,recoveries,tackled,tackles,target_missed,winning_goals
0,2016-17,Aaron Ramsey,0,0,4,0,2.7,16,181,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-17,Aaron Wan-Bissaka,0,0,0,0,0.0,612,181,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-17,Abdoulaye Doucouré,0,0,10,0,5.3,482,190,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-17,Ainsley Maitland-Niles,0,0,0,0,0.0,593,181,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-17,Alex Iwobi,0,3,37,1,62.2,21,181,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
cleaned_players_performances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170425 entries, 0 to 170424
Data columns (total 62 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   season_x                         170425 non-null  object 
 1   name                             170425 non-null  object 
 2   assists                          170425 non-null  int64  
 3   bonus                            170425 non-null  int64  
 4   bps                              170425 non-null  int64  
 5   clean_sheets                     170425 non-null  int64  
 6   creativity                       170425 non-null  float64
 7   element                          170425 non-null  int64  
 8   fixture                          170425 non-null  int64  
 9   goals_conceded                   170425 non-null  int64  
 10  goals_scored                     170425 non-null  int64  
 11  ict_index                        170425 non-null  float64
 12  in

In [252]:
cleaned_players_performances['position'].unique()

array(['MID', 'DEF', 'FWD', 'GK', nan, 'GKP'], dtype=object)

In [253]:
players_performances.isnull().sum()

season_x              0
name                  0
assists               0
bonus                 0
bps                   0
                  ...  
recoveries       148636
tackled          148636
tackles          148636
target_missed    148636
winning_goals    148636
Length: 62, dtype: int64

In [254]:
columns_to_drop = cleaned_players_performances.columns[cleaned_players_performances.isnull().sum() > 100000]
cleaned_players_performances = cleaned_players_performances.drop(columns=columns_to_drop)

In [255]:
cleaned_players_performances.shape

(170425, 38)

In [256]:
cleaned_players_performances = cleaned_players_performances.drop(columns={'position','team_a_score', 'team_h_score','kickoff_time'})

In [257]:
cleaned_players_performances.shape

(170425, 34)

In [258]:
cleaned_players_performances.isnull().sum()

season_x             0
name                 0
assists              0
bonus                0
bps                  0
clean_sheets         0
creativity           0
element              0
fixture              0
goals_conceded       0
goals_scored         0
ict_index            0
influence            0
minutes              0
opponent_team        0
own_goals            0
penalties_missed     0
penalties_saved      0
red_cards            0
round                0
saves                0
selected             0
threat               0
total_points         0
transfers_balance    0
transfers_in         0
transfers_out        0
value                0
was_home             0
yellow_cards         0
GW                   0
opp_team_name        0
Date                 0
team                 0
dtype: int64

In [259]:
cleaned_players_performances.shape

(170425, 34)

In [260]:
cleaned_players_performances.isnull().sum()

season_x             0
name                 0
assists              0
bonus                0
bps                  0
clean_sheets         0
creativity           0
element              0
fixture              0
goals_conceded       0
goals_scored         0
ict_index            0
influence            0
minutes              0
opponent_team        0
own_goals            0
penalties_missed     0
penalties_saved      0
red_cards            0
round                0
saves                0
selected             0
threat               0
total_points         0
transfers_balance    0
transfers_in         0
transfers_out        0
value                0
was_home             0
yellow_cards         0
GW                   0
opp_team_name        0
Date                 0
team                 0
dtype: int64

In [261]:
players_performances.shape

(170425, 62)

In [262]:
cleaned_players_performances.columns

Index(['season_x', 'name', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW', 'opp_team_name', 'Date', 'team'],
      dtype='object')

In [263]:
cleaned_players_performances['team_code'] = cleaned_players_performances['team'].map(team_to_id)

cleaned_players_performances['away_team_code'] = cleaned_players_performances['opp_team_name'].map(team_to_id)

In [264]:
cleaned_players_performances['was_home'] = cleaned_players_performances['was_home'].map({True: 1, False: 0})

In [265]:
cleaned_players_performances.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,transfers_out,value,was_home,yellow_cards,GW,opp_team_name,Date,team,team_code,away_team_code
0,2016-17,Aaron Ramsey,0,0,4,0,2.7,16,181,0,...,198,76,1,0,19,crystal palace,01/01/17,arsenal,1,9
1,2016-17,Aaron Wan-Bissaka,0,0,0,0,0.0,612,181,0,...,8,45,0,0,19,arsenal,01/01/17,crystal palace,9,1
2,2016-17,Abdoulaye Doucouré,0,0,10,0,5.3,482,190,4,...,10,46,1,0,19,tottenham,01/01/17,watford,30,29
3,2016-17,Ainsley Maitland-Niles,0,0,0,0,0.0,593,181,0,...,7,45,1,0,19,crystal palace,01/01/17,arsenal,1,9
4,2016-17,Alex Iwobi,0,3,37,1,62.2,21,181,0,...,1559,58,1,0,19,crystal palace,01/01/17,arsenal,1,9


In [266]:
# show the unique values in the 'name'
cleaned_players_performances['name'].nunique()

1884

In [267]:
# add each unique player unique code
unique_players = cleaned_players_performances['name'].unique()
unique_players_sorted = sorted(unique_players)

In [268]:
player_to_id = {player: idx for idx, player in enumerate(unique_players_sorted, start=1)}

In [269]:
player_to_id

{'Aaron Connolly': 1,
 'Aaron Cresswell': 2,
 'Aaron Hickey': 3,
 'Aaron Lennon': 4,
 'Aaron Mooy': 5,
 'Aaron Ramsdale': 6,
 'Aaron Ramsey': 7,
 'Aaron Rowe': 8,
 'Aaron Wan-Bissaka': 9,
 'Abd-Al-Ali Morakinyo Olaposi Koiki': 10,
 'Abdelhamid Sabiri': 11,
 'Abdoulaye Doucouré': 12,
 'Abdul Rahman Baba': 13,
 'Aboubakar Kamara': 14,
 'Abu Kamara': 15,
 'Adalberto Peñaranda': 16,
 'Adam Armstrong': 17,
 'Adam Davies': 18,
 'Adam Forshaw': 19,
 'Adam Idah': 20,
 'Adam Lallana': 21,
 'Adam Masina': 22,
 'Adam Smith': 23,
 'Adam Webster': 24,
 'Adam Wharton': 25,
 'Adama Diakhaby': 26,
 'Adama Traoré': 27,
 'Adama Traoré Diarra': 28,
 'Addji Keaninkin Marc-Israel Guehi': 29,
 'Adedapo Awokoya-Mebude': 30,
 'Ademipo Odubeko': 31,
 'Ademola Lookman': 32,
 'Ademola Ola-Adebomi': 33,
 'Admiral Muskwe': 34,
 'Adrian Blake': 35,
 'Adrian Mariappa': 36,
 'Adrien Sebastian Perruchet Silva': 37,
 'Adrien Silva': 38,
 'Adrián Bernabé': 39,
 'Adrián San Miguel del Castillo': 40,
 'Ahmed El-Sayed Hega

In [270]:
player_to_id_df = pd.DataFrame(player_to_id.items(), columns=['name', 'player_id'])

In [271]:
player_to_id_df.to_csv('data/player_to_id.csv', index=False)

In [272]:
cleaned_players_performances['player_code'] = cleaned_players_performances['name'].map(player_to_id)

In [273]:
cleaned_players_performances[cleaned_players_performances['player_code'] == 1]

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,value,was_home,yellow_cards,GW,opp_team_name,Date,team,team_code,away_team_code,player_code
43407,2019-20,Aaron Connolly,0,0,1,0,0.1,534,37,1,...,45,0,0,4,man city,31/08/19,brighton,5,18,1
43939,2019-20,Aaron Connolly,0,0,1,0,0.3,534,43,1,...,45,1,0,5,burnley,14/09/19,brighton,5,6,1
44478,2019-20,Aaron Connolly,0,0,1,0,4.8,534,58,0,...,45,0,0,6,newcastle,21/09/19,brighton,5,21,1
45019,2019-20,Aaron Connolly,0,0,2,0,0.6,534,63,1,...,45,0,0,7,chelsea,28/09/19,brighton,5,8,1
45565,2019-20,Aaron Connolly,0,3,53,1,23.8,534,72,0,...,45,1,0,8,tottenham,05/10/19,brighton,5,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167256,2023-24,Aaron Connolly,0,0,0,0,0.0,127,184,0,...,45,1,0,19,tottenham,28/12/23,brighton,5,29,1
167401,2023-24,Aaron Connolly,0,0,0,0,0.0,127,94,0,...,45,1,0,10,fulham,29/10/23,brighton,5,11,1
167761,2023-24,Aaron Connolly,0,0,0,0,0.0,127,214,0,...,45,0,0,22,luton,30/01/24,brighton,5,17,1
168849,2023-24,Aaron Connolly,0,0,0,0,0.0,127,61,0,...,45,0,0,7,aston villa,30/09/23,brighton,5,2,1


In [274]:
columns_to_keep = [
    'season_x',
    'team_code',    
    'away_team_code', 
    'was_home',  
    'influence',
    'Date',        
    'GW',                     
    'goals_scored', 
    'goals_conceded',
    'assists',
    'clean_sheets',
    'yellow_cards',
    'red_cards',
    'own_goals',
    'penalties_missed',
    'penalties_saved',
    'saves',
    'player_code'
]

cleaned_players_performances_filtered = cleaned_players_performances[columns_to_keep]

In [275]:
cleaned_players_performances_filtered.columns

Index(['season_x', 'team_code', 'away_team_code', 'was_home', 'influence',
       'Date', 'GW', 'goals_scored', 'goals_conceded', 'assists',
       'clean_sheets', 'yellow_cards', 'red_cards', 'own_goals',
       'penalties_missed', 'penalties_saved', 'saves', 'player_code'],
      dtype='object')

In [276]:
cleaned_players_performances_filtered.head()

,season_x,team_code,away_team_code,was_home,influence,Date,GW,goals_scored,goals_conceded,assists,clean_sheets,yellow_cards,red_cards,own_goals,penalties_missed,penalties_saved,saves,player_code
0,2016-17,1,9,1,1.4,01/01/17,19,0,0,0,0,0,0,0,0,0,0,7
1,2016-17,9,1,0,0.0,01/01/17,19,0,0,0,0,0,0,0,0,0,0,9
2,2016-17,30,29,1,3.6,01/01/17,19,0,4,0,0,0,0,0,0,0,0,12
3,2016-17,1,9,1,0.0,01/01/17,19,0,0,0,0,0,0,0,0,0,0,44
4,2016-17,1,9,1,55.6,01/01/17,19,1,0,0,1,0,0,0,0,0,0,54


In [277]:
cleaned_players_performances_filtered.dtypes

season_x             object
team_code             int64
away_team_code        int64
was_home              int64
influence           float64
Date                 object
GW                    int64
goals_scored          int64
goals_conceded        int64
assists               int64
clean_sheets          int64
yellow_cards          int64
red_cards             int64
own_goals             int64
penalties_missed      int64
penalties_saved       int64
saves                 int64
player_code           int64
dtype: object

In [278]:
cleaned_players_performances_filtered =cleaned_players_performances_filtered.rename(columns={'season_x': 'season'})

In [279]:
cleaned_players_performances_filtered = cleaned_players_performances_filtered.drop(columns= ['season', 'Date'])

In [280]:
cleaned_players_performances_filtered.head()

,team_code,away_team_code,was_home,influence,GW,goals_scored,goals_conceded,assists,clean_sheets,yellow_cards,red_cards,own_goals,penalties_missed,penalties_saved,saves,player_code
0,1,9,1,1.4,19,0,0,0,0,0,0,0,0,0,0,7
1,9,1,0,0.0,19,0,0,0,0,0,0,0,0,0,0,9
2,30,29,1,3.6,19,0,4,0,0,0,0,0,0,0,0,12
3,1,9,1,0.0,19,0,0,0,0,0,0,0,0,0,0,44
4,1,9,1,55.6,19,1,0,0,1,0,0,0,0,0,0,54


In [281]:
cleaned_players_performances_filtered.to_csv('data/encoded/encoded_players_performances.csv', index=False)